# Mount gdrive.

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Enter the foldername in your Drive.
FOLDERNAME = 'readmission'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Change directory.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
%cd drive/My\ Drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/My Drive/readmission


# Environment setup

In [ ]:
### Import packages.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import csv
import os
import tensorflow
import keras

In [ ]:
### Set up working directory.
os.chdir('/content/drive/MyDrive/readmission')
os.getcwd()

'/content/drive/MyDrive/readmission'

# (RUN ONCE) Get MIMIC-IV data.

In [ ]:
### Download zipped MIMIC-III data.
!wget -r -N -c -np --user kshi --ask-password https://physionet.org/files/mimiciv/1.0/


Password for user ‘kshi’: 
--2021-06-20 18:06:09--  https://physionet.org/files/mimiciv/1.0/
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 401 Unauthorized
Authentication selected: Basic realm="PhysioNet", charset="UTF-8"
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘physionet.org/files/mimiciv/1.0/index.html’

physionet.org/files     [ <=>                ]     797  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2021-06-20 18:06:09 (41.4 MB/s) - ‘physionet.org/files/mimiciv/1.0/index.html’ saved [797]

Loading robots.txt; please ignore errors.
--2021-06-20 18:06:09--  https://physionet.org/robots.txt
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 22 [text/plain]
Saving to: ‘physio

In [ ]:
### Unzip MIMIC-IV data. 
!mkdir -p mimic_database && gunzip *.gz

# Load files.

In [ ]:
admissions = pd.read_csv('admissions.csv', header = 0)
print(admissions.shape)
admissions.head()

(523740, 15)


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,14679932,21038362,2139-09-26 14:16:00,2139-09-28 11:30:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,SINGLE,UNKNOWN,NaN,NaN,0
1,15585972,24941086,2123-10-07 23:56:00,2123-10-12 11:22:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,WHITE,NaN,NaN,0
2,11989120,21965160,2147-01-14 09:00:00,2147-01-17 14:25:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,UNKNOWN,NaN,NaN,0
3,17817079,24709883,2165-12-27 17:33:00,2165-12-31 21:18:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,OTHER,NaN,NaN,0
4,15078341,23272159,2122-08-28 08:48:00,2122-08-30 12:32:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,0


In [ ]:
admissions.value_counts('admission_location')

admission_location
EMERGENCY ROOM                            245744
PHYSICIAN REFERRAL                        127494
TRANSFER FROM HOSPITAL                     39121
WALK-IN/SELF REFERRAL                      16660
CLINIC REFERRAL                            10670
PROCEDURE SITE                              8449
PACU                                        6067
INTERNAL TRANSFER TO OR FROM PSYCH          4467
TRANSFER FROM SKILLED NURSING FACILITY      4063
INFORMATION NOT AVAILABLE                    379
AMBULATORY SURGERY TRANSFER                  191
dtype: int64

In [ ]:
admissions.value_counts('discharge_location')

discharge_location
HOME                            222173
HOME HEALTH CARE                 86026
SKILLED NURSING FACILITY         46207
REHAB                            11328
DIED                              9238
CHRONIC/LONG TERM ACUTE CARE      7591
HOSPICE                           3679
ACUTE HOSPITAL                    3499
AGAINST ADVICE                    2718
PSYCH FACILITY                    2417
OTHER FACILITY                    1455
ASSISTED LIVING                    584
HEALTHCARE FACILITY                168
dtype: int64

In [ ]:
d_hcpcs = pd.read_csv('d_hcpcs.csv', header = 0)
print(d_hcpcs.shape)
d_hcpcs.head()

(89200, 4)


,code,category,long_description,short_description
0,A1,NaN,Dressing for one wound,Dressing for one wound
1,A2,NaN,Dressing for two wounds,Dressing for two wounds
2,A3,NaN,Dressing for three wounds,Dressing for three wounds
3,A4,NaN,Dressing for four wounds,Dressing for four wounds
4,A5,NaN,Dressing for five wounds,Dressing for five wounds


In [ ]:
d_icd_diagnoses = pd.read_csv('d_icd_diagnoses.csv', header = 0)
print(d_icd_diagnoses.shape)
d_icd_diagnoses.head()

(109775, 3)


,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A


In [ ]:
d_icd_procedures = pd.read_csv('d_icd_procedures.csv', header = 0)
print(d_icd_procedures.shape)
d_icd_procedures.head()

(85257, 3)


,icd_code,icd_version,long_title
0,0001,9,Therapeutic ultrasound of vessels of head and ...
1,0002,9,Therapeutic ultrasound of heart
2,0003,9,Therapeutic ultrasound of peripheral vascular ...
3,0009,9,Other therapeutic ultrasound
4,001,10,"Central Nervous System and Cranial Nerves, Bypass"


In [ ]:
d_items = pd.read_csv('d_items.csv', header = 0)
print(d_items.shape)
d_items.head()


(3861, 9)


,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN
1,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
2,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
3,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220048,Heart Rhythm,Heart Rhythm,chartevents,Routine Vital Signs,NaN,Text,NaN,NaN


In [ ]:
d_labitems = pd.read_csv('d_labitems.csv', header = 0)
print(d_labitems.shape)
d_labitems.head()


(1630, 5)


,itemid,label,fluid,category,loinc_code
0,51905,,Other Body Fluid,Chemistry,NaN
1,51532,11-Deoxycorticosterone,Blood,Chemistry,NaN
2,51957,17-Hydroxycorticosteroids,Urine,Chemistry,NaN
3,51958,"17-Ketosteroids, Urine",Urine,Chemistry,NaN
4,52068,24 Hr,Blood,Hematology,NaN


In [ ]:
datetimeevents = pd.read_csv('datetimeevents.csv', header=0)
print(datetimeevents.shape)
datetimeevents.head()

(7495712, 9)


,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valueuom,warning
0,10003700,28623837,30600691,2165-04-24 05:42:00,2165-04-24 05:42:00,225755,2165-04-24 05:42:00,Date,0
1,10003700,28623837,30600691,2165-04-24 08:00:00,2165-04-24 08:26:00,225755,2165-04-24 00:00:00,Date,0
2,10004235,24181354,34100191,2196-02-24 18:06:00,2196-02-24 18:07:00,224261,2196-02-24 18:06:00,Date,0
3,10004235,24181354,34100191,2196-02-24 18:06:00,2196-02-24 18:07:00,224279,2196-02-24 18:06:00,Date and Time,0
4,10004235,24181354,34100191,2196-02-24 18:06:00,2196-02-24 18:07:00,224280,2196-02-24 18:06:00,Date,0


In [ ]:
diagnoses_icd = pd.read_csv('diagnoses_icd.csv', header=0)
print(diagnoses_icd.shape)
diagnoses_icd.head()

(5280351, 5)


,subject_id,hadm_id,seq_num,icd_code,icd_version
0,15734973,20475282,3,2825,9
1,15734973,20475282,2,V0251,9
2,15734973,20475282,5,V270,9
3,15734973,20475282,1,64891,9
4,15734973,20475282,4,66481,9


In [ ]:
drgcodes = pd.read_csv('drgcodes.csv', header=0)
print(drgcodes.shape)
drgcodes.head()

(769622, 7)


,subject_id,hadm_id,drg_type,drg_code,description,drg_severity,drg_mortality
0,15734973,20475282,HCFA,775,VAGINAL DELIVERY W/O COMPLICATING DIAGNOSES,NaN,NaN
1,11442057,21518990,HCFA,765,CESAREAN SECTION W CC/MCC,NaN,NaN
2,10072949,20817034,HCFA,766,CESAREAN SECTION W/O CC/MCC,NaN,NaN
3,13538405,24822466,HCFA,775,VAGINAL DELIVERY W/O COMPLICATING DIAGNOSES,NaN,NaN
4,10287061,27485182,HCFA,775,VAGINAL DELIVERY W/O COMPLICATING DIAGNOSES,NaN,NaN


In [ ]:
hcpcsevents = pd.read_csv('hcpcsevents.csv', header=0)
print(hcpcsevents.shape)
hcpcsevents.head()

(160727, 6)


,subject_id,hadm_id,chartdate,hcpcs_cd,seq_num,short_description
0,12350009,25467647,2155-02-10,99219,1,Hospital observation services
1,14047846,22391731,2120-10-29,99218,2,Hospital observation services
2,14047846,22391731,2120-10-29,54401,1,Male genital system
3,12854593,21980916,2157-11-01,11622,1,Integumentary system
4,12854593,21980916,2157-11-01,12041,2,Integumentary system


In [ ]:
icustays = pd.read_csv('icustays.csv', header=0)
print(icustays.shape)
icustays.head()

(76540, 8)


,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,17867402,24528534,31793211,Trauma SICU (TSICU),Trauma SICU (TSICU),2154-03-03 04:11:00,2154-03-04 18:16:56,1.587454
1,14435996,28960964,31983544,Trauma SICU (TSICU),Trauma SICU (TSICU),2150-06-19 17:57:00,2150-06-22 18:33:54,3.025625
2,17609946,27385897,33183475,Trauma SICU (TSICU),Trauma SICU (TSICU),2138-02-05 18:54:00,2138-02-15 12:42:05,9.741725
3,18966770,23483021,34131444,Trauma SICU (TSICU),Trauma SICU (TSICU),2123-10-25 10:35:00,2123-10-25 18:59:47,0.350544
4,12776735,20817525,34547665,Neuro Stepdown,Neuro Stepdown,2200-07-12 00:33:00,2200-07-13 16:44:40,1.674769


In [ ]:
icustays.value_counts('last_careunit')

last_careunit
Medical Intensive Care Unit (MICU)                  17717
Medical/Surgical Intensive Care Unit (MICU/SICU)    13015
Cardiac Vascular Intensive Care Unit (CVICU)        11950
Surgical Intensive Care Unit (SICU)                 11615
Coronary Care Unit (CCU)                             8830
Trauma SICU (TSICU)                                  8690
Neuro Intermediate                                   2417
Neuro Stepdown                                       1226
Neuro Surgical Intensive Care Unit (Neuro SICU)      1080
dtype: int64

In [ ]:
inputevents = pd.read_csv('inputevents.csv', header=0)
print(inputevents.shape)
inputevents.head()

(9460658, 26)


,subject_id,hadm_id,stay_id,starttime,endtime,storetime,itemid,amount,amountuom,rate,rateuom,orderid,linkorderid,ordercategoryname,secondaryordercategoryname,ordercomponenttypedescription,ordercategorydescription,patientweight,totalamount,totalamountuom,isopenbag,continueinnextdept,cancelreason,statusdescription,originalamount,originalrate
0,12481680,26876606,30863119,2141-01-23 17:37:00,2141-01-23 17:38:00,2141-01-23 17:37:00,226452,180.0,ml,NaN,NaN,4988314,4988314,14-Oral/Gastric Intake,NaN,Main order parameter,Bolus,123.0,180.0,ml,0,0,0,FinishedRunning,180.0,180.0
1,12481680,26876606,30863119,2141-01-23 13:00:00,2141-01-23 13:01:00,2141-01-23 13:23:00,226452,240.0,ml,NaN,NaN,6199041,6199041,14-Oral/Gastric Intake,NaN,Main order parameter,Bolus,123.0,240.0,ml,0,0,0,FinishedRunning,240.0,240.0
2,12481680,26876606,30863119,2141-01-23 11:00:00,2141-01-23 11:01:00,2141-01-23 12:24:00,226452,120.0,ml,NaN,NaN,8638288,8638288,14-Oral/Gastric Intake,NaN,Main order parameter,Bolus,123.0,120.0,ml,0,0,0,FinishedRunning,120.0,120.0
3,12481680,26876606,30863119,2141-01-23 14:16:00,2141-01-23 14:17:00,2141-01-23 14:17:00,226452,60.0,ml,NaN,NaN,9238040,9238040,14-Oral/Gastric Intake,NaN,Main order parameter,Bolus,123.0,60.0,ml,0,0,0,FinishedRunning,60.0,60.0
4,15614172,27424463,33484414,2153-07-30 23:16:00,2153-07-30 23:17:00,2153-07-30 23:16:00,226452,100.0,ml,NaN,NaN,7034770,7034770,14-Oral/Gastric Intake,NaN,Main order parameter,Bolus,71.6,100.0,ml,0,0,0,FinishedRunning,100.0,100.0


In [ ]:
microbiologyevents = pd.read_csv('microbiologyevents.csv', header=0)
print(microbiologyevents.shape)
microbiologyevents.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(3397914, 24)


,microevent_id,subject_id,hadm_id,micro_specimen_id,chartdate,charttime,spec_itemid,spec_type_desc,test_seq,storedate,storetime,test_itemid,test_name,org_itemid,org_name,isolate_num,quantity,ab_itemid,ab_name,dilution_text,dilution_comparison,dilution_value,interpretation,comments
0,97,10000473,NaN,5491242,2138-03-15 00:00:00,2138-03-15 22:37:00,70012,BLOOD CULTURE,1,2138-03-16 00:00:00,2138-03-16 05:10:00,90201,"Blood Culture, Routine",90760.0,CANCELLED,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,457,10001319,NaN,2654897,2135-06-10 00:00:00,2135-06-10 11:19:00,70079,URINE,1,2135-06-11 00:00:00,2135-06-11 21:02:00,90039,URINE CULTURE,80081.0,GRAM POSITIVE BACTERIA,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,634,10001757,NaN,3685553,2171-06-10 00:00:00,2171-06-10 10:18:00,70079,URINE,1,2171-06-12 00:00:00,2171-06-12 14:00:00,90039,URINE CULTURE,80002.0,ESCHERICHIA COLI,1.0,NaN,90004.0,AMPICILLIN,<=2,<=,2.0,S,NaN
3,635,10001757,NaN,3685553,2171-06-10 00:00:00,2171-06-10 10:18:00,70079,URINE,1,2171-06-12 00:00:00,2171-06-12 14:00:00,90039,URINE CULTURE,80002.0,ESCHERICHIA COLI,1.0,NaN,90005.0,CEFAZOLIN,<=4,<=,4.0,S,NaN
4,636,10001757,NaN,3685553,2171-06-10 00:00:00,2171-06-10 10:18:00,70079,URINE,1,2171-06-12 00:00:00,2171-06-12 14:00:00,90039,URINE CULTURE,80002.0,ESCHERICHIA COLI,1.0,NaN,90008.0,TRIMETHOPRIM/SULFA,<=1,<=,1.0,S,NaN


In [ ]:
outputevents = pd.read_csv('outputevents.csv', header=0)
print(outputevents.shape)
outputevents.head()

(4457381, 8)


,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valueuom
0,10003700,28623837,30600691,2165-04-24 05:40:00,2165-04-24 05:44:00,226559,300.0,ml
1,10004235,24181354,34100191,2196-02-24 17:55:00,2196-02-24 17:55:00,226559,100.0,ml
2,10004235,24181354,34100191,2196-02-24 19:00:00,2196-02-24 19:43:00,226559,45.0,ml
3,10004235,24181354,34100191,2196-02-24 20:00:00,2196-02-24 20:02:00,226559,45.0,ml
4,10004235,24181354,34100191,2196-02-24 21:00:00,2196-02-24 20:56:00,226559,45.0,ml


In [ ]:
patients = pd.read_csv('patients.csv', header=0)
print(patients.shape)
patients.head()

(382278, 6)


,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000048,F,23,2126,2008 - 2010,NaN
1,10002723,F,0,2128,2017 - 2019,NaN
2,10003939,M,0,2184,2008 - 2010,NaN
3,10004222,M,0,2161,2014 - 2016,NaN
4,10005325,F,0,2154,2011 - 2013,NaN


In [ ]:
### Limited RAM left on this (takes 8GB).
pharmacy = pd.read_csv('pharmacy.csv', header=0)
print(pharmacy.shape)
pharmacy.head()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,18,24,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(14736386, 27)


,subject_id,hadm_id,pharmacy_id,poe_id,starttime,stoptime,medication,proc_type,status,entertime,verifiedtime,route,frequency,disp_sched,infusion_type,sliding_scale,lockout_interval,basal_rate,one_hr_max,doses_per_24_hrs,duration,duration_interval,expiration_value,expiration_unit,expirationdate,dispensation,fill_quantity
0,17868682,22726960,2697460,17868682-19,2160-01-07 14:00:00,2160-01-07 14:00:00,NaN,IV Large Volume,Discontinued,2160-01-07 13:06:18,2160-01-07 13:06:18,IV,ASDIR,NaN,C,N,NaN,NaN,NaN,NaN,NaN,Ongoing,NaN,Enter on Label,NaN,Distribution-Floor Stock,NaN
1,17868682,22726960,1383959,17868682-20,2160-01-07 15:00:00,2160-01-08 16:00:00,Pneumococcal Vac Polyvalent,Unit Dose,Discontinued via patient discharge,2160-01-07 14:25:45,2160-01-07 14:25:45,IM,ASDIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,Omnicell,NaN
2,17868682,22726960,64138666,17868682-23,2160-01-07 15:00:00,2160-01-08 16:00:00,Sodium Chloride 0.9% Flush,Unit Dose,Discontinued via patient discharge,2160-01-07 14:25:45,2160-01-07 14:25:45,IV,Q8H:PRN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,Floor Stock Item,NaN
3,17868682,22726960,43535627,17868682-28,2160-01-07 15:00:00,2160-01-08 14:00:00,Ondansetron,Unit Dose,Expired,2160-01-07 14:25:45,2160-01-07 14:25:45,IV,ONCE,15,NaN,NaN,NaN,NaN,NaN,1.0,1.0,Doses,36.0,Hours,NaN,Omnicell,NaN
4,17868682,22726960,13072826,17868682-29,2160-01-07 15:00:00,2160-01-07 16:00:00,Morphine Sulfate,Unit Dose,Discontinued,2160-01-07 14:25:45,2160-01-07 14:25:45,IV,Q4H:PRN,NaN,NaN,NaN,6,0.0,NaN,NaN,NaN,Ongoing,36.0,Hours,NaN,Omnicell,NaN


In [ ]:
poe_detail = pd.read_csv('poe_detail.csv', header=0)
print(poe_detail.shape)
poe_detail.head()

(3256358, 5)


,poe_id,poe_seq,subject_id,field_name,field_value
0,17741087-177,177,17741087,Admit to,ED - Observation
1,17385589-1252,1252,17385589,Admit to,ED - Observation
2,16865432-194,194,16865432,Admit to,ED - Observation
3,18948691-737,737,18948691,Admit to,MICU - Red
4,18600838-131,131,18600838,Admit to,ED - Observation


In [ ]:
prescriptions = pd.read_csv('prescriptions.csv', header=0)
print(prescriptions.shape)
prescriptions.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(17008053, 17)


,subject_id,hadm_id,pharmacy_id,starttime,stoptime,drug_type,drug,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,17868682,22726960,73313910,2160-01-07 08:00:00,2160-01-07 21:00:00,MAIN,BuPROPion (Sustained Release),046238,591083960.0,150mg SR Tablet,NaN,300,mg,2,TAB,1.0,PO
1,17868682,22726960,16239987,2160-01-07 08:00:00,2160-01-08 16:00:00,MAIN,BuPROPion (Sustained Release),046238,591083960.0,150mg SR Tablet,NaN,150,mg,1,TAB,1.0,PO
2,17868682,22726960,16634804,2160-01-07 10:00:00,2160-01-07 16:00:00,MAIN,Aspirin,004380,904404073.0,81mg Tab,NaN,81,mg,1,TAB,1.0,PO
3,17868682,22726960,2697460,2160-01-07 14:00:00,2160-01-07 14:00:00,BASE,1/2 NS,001209,338004304.0,1000mL Bag,NaN,1000,mL,1000,mL,NaN,IV
4,17868682,22726960,1383959,2160-01-07 15:00:00,2160-01-08 16:00:00,MAIN,Pneumococcal Vac Polyvalent,048548,6494300.0,25mcg/0.5mL Vial,NaN,0.5,mL,1,VIAL,NaN,IM


In [ ]:
procedureevents = pd.read_csv('procedureevents.csv', header=0)
print(procedureevents.shape)
procedureevents.head()

(731247, 26)


,subject_id,hadm_id,stay_id,starttime,endtime,storetime,itemid,value,valueuom,location,locationcategory,orderid,linkorderid,ordercategoryname,secondaryordercategoryname,ordercategorydescription,patientweight,totalamount,totalamountuom,isopenbag,continueinnextdept,cancelreason,statusdescription,comments_date,originalamount,originalrate
0,15693895,21203492,30500789,2147-06-05 11:25:00,2147-06-05 11:26:00,2147-06-06 11:25:00,225399,1.0,None,NaN,NaN,6591564,6591564,Procedures,NaN,Task,80.0,NaN,NaN,0,0,0,FinishedRunning,NaN,1.0,0
1,15693895,21203492,30500789,2147-06-05 22:00:00,2147-06-05 22:01:00,2147-06-06 11:25:00,224385,1.0,None,NaN,NaN,1706064,1706064,Intubation/Extubation,NaN,Task,80.0,NaN,NaN,0,0,0,FinishedRunning,NaN,1.0,0
2,12481680,26876606,30863119,2141-01-23 12:00:00,2141-01-23 18:33:00,2141-01-23 18:33:51.14,224275,393.0,min,NaN,NaN,8373796,8373796,Peripheral Lines,NaN,ContinuousProcess,123.0,NaN,NaN,1,0,0,FinishedRunning,NaN,393.0,1
3,12481680,26876606,30863119,2141-01-23 12:00:00,2141-01-23 18:33:00,2141-01-23 18:33:51.14,224275,393.0,min,NaN,NaN,3553081,3553081,Peripheral Lines,NaN,ContinuousProcess,123.0,NaN,NaN,1,0,0,FinishedRunning,NaN,393.0,1
4,15614172,27424463,33484414,2153-07-29 18:13:00,2153-07-31 01:25:00,2153-07-31 01:26:00,224277,1872.0,min,RL Post Forearm,Peripheral - old,137889,137889,Peripheral Lines,NaN,ContinuousProcess,71.6,NaN,NaN,1,1,0,FinishedRunning,NaN,1872.0,1


In [ ]:
procedures_icd = pd.read_csv('procedures_icd.csv', header=0)
print(procedures_icd.shape)
procedures_icd.head()

(779625, 6)


,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version
0,10287061,27485182,1,2124-08-22,7569,9
1,13135573,22286210,3,2187-05-30,7309,9
2,13135573,22286210,1,2187-05-30,7569,9
3,13135573,22286210,2,2187-05-30,734,9
4,12646116,24154012,2,2115-01-13,734,9


In [ ]:
services = pd.read_csv('services.csv', header=0)
print(services.shape)
services.head()

(562892, 5)


,subject_id,hadm_id,transfertime,prev_service,curr_service
0,10417172,29884156,2163-12-09 01:23:24,NaN,MED
1,14299936,27900446,2182-02-13 21:45:00,NaN,MED
2,18848104,20140684,2124-05-24 00:03:32,NaN,TRAUM
3,19295127,26612830,2116-04-21 16:33:54,NaN,OBS
4,11327174,23934262,2159-01-26 17:31:32,NaN,MED


In [ ]:
transfers = pd.read_csv('transfers.csv', header=0)
print(transfers.shape)
transfers.head()

(2189535, 7)


,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,14550633,24512724.0,35906369,admit,NaN,2176-10-10 01:58:40,2176-10-10 10:00:30
1,15216953,25759433.0,37381511,admit,NaN,2173-06-14 00:08:04,2173-06-14 13:35:19
2,15776555,28490372.0,34638843,admit,NaN,2187-02-01 00:09:41,2187-02-01 14:25:07
3,12248661,23426497.0,39272104,discharge,NaN,2141-12-05 14:11:09,NaN
4,14261068,20086032.0,36182135,discharge,NaN,2144-09-13 14:14:50,NaN


In [ ]:
transfers.value_counts('eventtype')


eventtype
ED           661053
admit        523749
discharge    523740
transfer     480993
dtype: int64

In [ ]:
transfers.value_counts('careunit')

careunit
Emergency Department                                661053
Medicine                                            150908
Emergency Department Observation                     85815
Nursery - Well Babies                                67664
Med/Surg                                             49926
Medicine/Cardiology                                  46447
Neurology                                            41183
Hematology/Oncology                                  37522
Vascular                                             32965
Transplant                                           31529
Neonatal Intensive Care Unit (NICU)                  29363
Med/Surg/Trauma                                      28112
Med/Surg/GYN                                         27993
PACU                                                 26459
Labor & Delivery                                     24038
Medical Intensive Care Unit (MICU)                   22314
Surgery/Trauma                                 

In [ ]:
chartevents = pd.read_csv('chartevents.csv', header=0)
print(chartevents.shape)
chartevents.head()

In [ ]:
### Limited RAM left on this (takes 7GB).
emar = pd.read_csv('emar.csv', header=0)
print(emar.shape)
emar.head()

(27464367, 11)


,subject_id,hadm_id,emar_id,emar_seq,poe_id,pharmacy_id,charttime,medication,event_txt,scheduletime,storetime
0,10000473,NaN,10000473-15,15,10000473-23,NaN,2138-03-16 03:53:00,Doxycycline Hyclate,Administered,2138-03-16 03:53:00,2138-03-16 03:53:00
1,10000764,NaN,10000764-17,17,10000764-9,NaN,2132-10-14 21:49:00,Amoxicillin-Clavulanic Acid,Administered,2132-10-14 21:49:00,2132-10-14 21:53:00
2,10000764,NaN,10000764-18,18,10000764-8,NaN,2132-10-14 21:49:00,Tetanus-DiphTox-Acellular Pertuss (Adacel),Administered,2132-10-14 21:49:00,2132-10-14 21:53:00
3,10000764,27897940.0,10000764-19,19,10000764-11,NaN,2132-10-14 23:54:00,PNEUMOcoccal 23-valent polysaccharide vaccine,Not Given,2132-10-14 23:54:00,2132-10-14 23:55:00
4,10000764,27897940.0,10000764-20,20,10000764-10,NaN,2132-10-14 23:55:00,Influenza Vaccine Quadrivalent,Not Given,2132-10-14 23:55:00,2132-10-14 23:55:00


In [ ]:
### Need to parse into separate files.
labevents = pd.read_csv('labevents.csv', header=0)
print(labevents.shape)
labevents.head()

In [ ]:
### Need to parse into separate files. 
emar_detail = pd.read_csv('emar_detail.csv', header=0)
print(emar_detail.shape)
emar_detail.head()

In [ ]:
### Need to parse into separate files. Cannot load at all. 
poe = pd.read_csv('poe.csv', header=0)
print(poe.shape)
poe.head()

# COHORT

In [ ]:
cohort = pd.read_csv('cohort.csv', header = 0)
print(cohort.shape)
cohort.head()

In [ ]:
from datetime import datetime

cohort['readmit'] = np.where((cohort.no_icustays == 1), 0, 1)
cohort['admit_ed'] = np.where((cohort.admission_location == "EMERGENCY ROOM"), 1, 0)

hosp_len = []
for i in range(cohort.shape[0]):
  transfertime = datetime.strptime(cohort['outtime'][i], '%Y-%m-%d %H:%M:%S').day
  admittime = datetime.strptime(cohort['admittime'][i], '%Y-%m-%d %H:%M:%S').day
  hosp_len.append(transfertime - admittime)

cohort['hospital_stay'] = hosp_len

cohort.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,admittime,dischtime,admission_location,discharge_location,insurance,ethnicity,no_icustays,gender,anchor_age,anchor_year,anchor_year_group,dod,readmit,admit_ed,hospital_stay
0,15175657,24212233,30725529,Neuro Stepdown,Neuro Stepdown,2114-10-23 04:24:00,2114-10-24 16:30:34,1.504560,2114-10-22 00:00:00,2114-10-26 14:45:00,PHYSICIAN REFERRAL,HOME,Other,HISPANIC/LATINO,1,M,18,2114,2014 - 2016,NaN,0,0,2
1,17843231,25996300,39919230,Surgical Intensive Care Unit (SICU),Neuro Stepdown,2157-11-23 16:02:36,2157-11-24 23:39:22,1.317199,2157-11-23 16:02:00,2157-11-24 23:30:00,EMERGENCY ROOM,HOME,Medicaid,WHITE,1,M,19,2157,2014 - 2016,NaN,0,1,1
2,17384222,26475614,34018090,Trauma SICU (TSICU),Neuro Stepdown,2111-03-12 21:43:00,2111-03-21 12:06:09,8.599410,2111-03-12 21:05:00,2111-03-22 13:10:00,EMERGENCY ROOM,HOME,Other,WHITE,1,M,20,2111,2017 - 2019,NaN,0,1,9
3,17406955,26123136,31860020,Neuro Stepdown,Neuro Stepdown,2137-10-25 05:40:00,2137-10-29 15:55:09,4.427187,2137-10-25 03:37:00,2137-10-29 15:30:00,EMERGENCY ROOM,HOME,Other,OTHER,1,F,20,2137,2017 - 2019,NaN,0,1,4
4,10816811,25685438,39565076,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Stepdown,2146-08-17 12:16:15,2146-08-20 11:55:49,2.985810,2146-08-10 05:43:00,2146-08-24 15:15:00,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,WHITE,1,F,20,2138,2008 - 2010,NaN,0,0,10


In [ ]:
cohort.value_counts('hospital_stay')

hospital_stay
2     9647
1     8069
3     5749
4     3664
5     2549
      ... 
26      13
27       7
29       4
28       4
30       1
Length: 61, dtype: int64

In [ ]:
cohort_final = pd.merge(cohort, labs, how = 'outer', on = ['subject_id', 'hadm_id', 'stay_id'])
print(cohort_final.shape)
cohort_final.head()


(44245, 165)


,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,admittime,dischtime,admission_location,discharge_location,insurance,ethnicity,no_icustays,gender,anchor_age,anchor_year,anchor_year_group,dod,readmit,admit_ed,hospital_stay,alt_min,ast_min,alkphos_min,ammonia_min,anion_gap_min,bicarb_min,bnp_min,bun_min,cl_min,ck_min,cortisol_min,cr_min,crp_min,dbili_min,esr_min,fio2_min,glu_min,...,map_last,na_last,pH_last,paco2_last,pao2_last,plt_last,rr_last,sao2_last,spo2_last,tbili_last,temp_last,transfusion_consent,tropT_last,wbc_last,wgt,hco3_high,bnp_high,bun_high,crp_high,esr_high,glu_high,hr_high,k_high,lac_high,map_high,na_high,rr_high,temp_high,tropT_high,wbc_high,hco3_low,hgb_low,hr_low,k_low,map_low,na_low,plt_low,rr_low,temp_low,wbc_low
0,15175657,24212233,30725529,Neuro Stepdown,Neuro Stepdown,2114-10-23 04:24:00,2114-10-24 16:30:34,1.504560,2114-10-22 00:00:00,2114-10-26 14:45:00,PHYSICIAN REFERRAL,HOME,Other,HISPANIC/LATINO,1,M,18,2114,2014 - 2016,NaN,0,0,2,NaN,NaN,NaN,NaN,19.0,22.0,NaN,11.0,100.0,NaN,NaN,0.6,NaN,NaN,NaN,NaN,100.0,...,NaN,137.0,NaN,NaN,NaN,303.0,12.0,NaN,100.0,NaN,98.4,NaN,NaN,20.3,89.5,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,17843231,25996300,39919230,Surgical Intensive Care Unit (SICU),Neuro Stepdown,2157-11-23 16:02:36,2157-11-24 23:39:22,1.317199,2157-11-23 16:02:00,2157-11-24 23:30:00,EMERGENCY ROOM,HOME,Medicaid,WHITE,1,M,19,2157,2014 - 2016,NaN,0,1,1,NaN,NaN,NaN,NaN,16.0,27.0,NaN,13.0,98.0,NaN,NaN,0.9,NaN,NaN,NaN,NaN,71.0,...,NaN,137.0,NaN,NaN,NaN,233.0,16.0,NaN,100.0,NaN,98.4,NaN,NaN,6.1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17384222,26475614,34018090,Trauma SICU (TSICU),Neuro Stepdown,2111-03-12 21:43:00,2111-03-21 12:06:09,8.599410,2111-03-12 21:05:00,2111-03-22 13:10:00,EMERGENCY ROOM,HOME,Other,WHITE,1,M,20,2111,2017 - 2019,NaN,0,1,9,NaN,NaN,NaN,NaN,16.0,24.0,NaN,9.0,94.0,NaN,NaN,0.6,NaN,NaN,NaN,NaN,90.0,...,NaN,133.0,NaN,NaN,NaN,371.0,11.0,NaN,100.0,NaN,98.8,NaN,NaN,10.2,NaN,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0
3,17406955,26123136,31860020,Neuro Stepdown,Neuro Stepdown,2137-10-25 05:40:00,2137-10-29 15:55:09,4.427187,2137-10-25 03:37:00,2137-10-29 15:30:00,EMERGENCY ROOM,HOME,Other,OTHER,1,F,20,2137,2017 - 2019,NaN,0,1,4,188.0,134.0,44.0,NaN,13.0,24.0,NaN,11.0,97.0,NaN,NaN,0.6,NaN,NaN,NaN,NaN,98.0,...,NaN,136.0,NaN,NaN,NaN,201.0,21.0,NaN,97.0,0.3,98.3,NaN,NaN,2.8,NaN,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,10816811,25685438,39565076,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Stepdown,2146-08-17 12:16:15,2146-08-20 11:55:49,2.985810,2146-08-10 05:43:00,2146-08-24 15:15:00,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,WHITE,1,F,20,2138,2008 - 2010,NaN,0,0,10,NaN,NaN,NaN,NaN,14.0,21.0,NaN,8.0,98.0,NaN,NaN,0.4,NaN,NaN,NaN,NaN,110.0,...,NaN,134.0,NaN,NaN,NaN,401.0,18.0,NaN,95.0,NaN,97.7,NaN,NaN,14.8,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


# LABS

In [ ]:
labs_counts_low = pd.read_csv('labs_icu_counts_low.csv', header = 0)
print(labs_counts_low.shape)
labs_counts_low.head()

(620277, 16)


,subject_id,hadm_id,stay_id,charttime,value,valuenum,valueuom,feature,item_id,intime,outtime,los,ethnicity,no_icustays,gender,anchor_age
0,10001884,26184834,37510196,2131-01-13 03:00:00,50.0,50.0,bpm,hr,220045,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,BLACK/AFRICAN AMERICAN,1,F,68
1,10001884,26184834,37510196,2131-01-17 13:49:00,7.7,7.7,g/dl,hgb,220228,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,BLACK/AFRICAN AMERICAN,1,F,68
2,10001884,26184834,37510196,2131-01-13 04:00:00,0.0,0.0,insp/min,rr,224689,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,BLACK/AFRICAN AMERICAN,1,F,68
3,10001884,26184834,37510196,2131-01-11 07:00:00,0.0,0.0,insp/min,rr,224689,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,BLACK/AFRICAN AMERICAN,1,F,68
4,10001884,26184834,37510196,2131-01-11 11:00:00,0.0,0.0,insp/min,rr,224689,2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,BLACK/AFRICAN AMERICAN,1,F,68


In [ ]:
labs_low = labs_counts_low.pivot_table('valuenum', ['subject_id', 'hadm_id', 'stay_id'], 'feature', aggfunc = 'count')

labs_low.reset_index(drop = False, inplace = True)
labs_low = labs_low.rename_axis(None, axis=1)
#labs_low.drop(labs_low.columns[[19, 20, 21]], axis = 1, inplace = True)
labs_low.columns = ['subject_id', 'hadm_id', 'stay_id', 'hco3_low', 'hgb_low', 'hr_low', 'k_low', 'map_low', 'na_low', 'plt_low', 'rr_low', 'temp_low', 'wbc_low']
labs_low = labs_low.fillna(0)
print(labs_low.shape)
labs_low.head()

(38543, 13)


,subject_id,hadm_id,stay_id,hco3_low,hgb_low,hr_low,k_low,map_low,na_low,plt_low,rr_low,temp_low,wbc_low
0,10001884,26184834,37510196,0.0,6.0,8.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0
1,10002013,23581541,39060235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,10002155,23822395,33685454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2.0
3,10002223,22494570,39638202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,10002348,22725460,32610785,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
labs_low.nunique() # wo count

subject_id    38543
hadm_id       38543
stay_id       38543
bicarb          829
hgb            1383
hr             1302
k                50
map               2
na              556
plt            2319
rr             2871
temp           1961
wbc             683
dtype: int64

In [ ]:
labs_low.nunique() # w count 

subject_id    38543
hadm_id       38543
stay_id       38543
bicarb           65
hgb              64
hr              115
k                 8
map               1
na               52
plt              82
rr              200
temp             93
wbc              39
dtype: int64

In [ ]:
labs_low.describe()

,subject_id,hadm_id,stay_id,bicarb,hgb,hr,k,map,na,plt,rr,temp,wbc
count,3.854300e+04,3.854300e+04,3.854300e+04,16024.000000,11049.000000,6548.000000,484.000000,2.0,4181.000000,8846.000000,26812.000000,13785.000000,3320.000000
mean,1.499681e+07,2.498697e+07,3.497651e+07,3.969920,4.265001,8.259774,1.338843,1.0,4.310691,6.210943,11.382217,4.806094,3.134639
std,2.884946e+06,2.881812e+06,2.894562e+06,5.270914,5.664434,15.701182,1.365373,0.0,6.438955,8.751544,27.824718,8.003575,4.275776
min,1.000188e+07,2.000015e+07,3.000015e+07,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.250581e+07,2.250277e+07,3.245384e+07,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,2.000000,2.000000,1.000000,1.000000
50%,1.499689e+07,2.496742e+07,3.495263e+07,2.000000,2.000000,3.000000,1.000000,1.0,2.000000,3.000000,4.000000,2.000000,2.000000
75%,1.750424e+07,2.747766e+07,3.748175e+07,5.000000,5.000000,9.000000,1.000000,1.0,5.000000,7.000000,11.000000,5.000000,4.000000
max,1.999999e+07,2.999962e+07,3.999955e+07,105.000000,94.000000,385.000000,25.000000,1.0,89.000000,180.000000,2566.000000,235.000000,93.000000


In [ ]:
labs_counts_high = pd.read_csv('labs_icu_counts_high.csv', header = 0)
print(labs_counts_high.shape)
labs_counts_high.head()

(3841920, 6)


,subject_id,hadm_id,stay_id,feature,valuenum,valueuom
0,10002155,23822395,33685454,tropT,2.65,ng/mL
1,10002155,23822395,33685454,tropT,3.99,ng/mL
2,10002430,26295318,38392119,bnp,23132.00,pg/mL
3,10002495,24982426,36753294,glu,370.00,mg/dL
4,10002495,24982426,36753294,glu,331.00,mg/dL


In [ ]:
labs_high = labs_counts_high.pivot_table('valuenum', ['subject_id', 'hadm_id', 'stay_id'], 'feature', aggfunc = 'count')

labs_high.reset_index(drop = False, inplace = True)
labs_high = labs_high.rename_axis(None, axis=1)
#labs_low.drop(labs_low.columns[[19, 20, 21]], axis = 1, inplace = True)
labs_high.columns = ['subject_id', 'hadm_id', 'stay_id', 'hco3_high', 'bnp_high', 'bun_high', 'crp_high', 'esr_high', 'glu_high', 'hr_high', 'k_high', 'lac_high', 'map_high', 'na_high', 'rr_high', 'temp_high', 'tropT_high', 'wbc_high']
labs_high = labs_high.fillna(0)
print(labs_high.shape)
labs_high.head()

(44095, 18)


,subject_id,hadm_id,stay_id,hco3_high,bnp_high,bun_high,crp_high,esr_high,glu_high,hr_high,k_high,lac_high,map_high,na_high,rr_high,temp_high,tropT_high,wbc_high
0,10001217,24597018,37067082,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,1.0
1,10001725,25563031,31205490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,2.0
2,10001884,26184834,37510196,14.0,0.0,11.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,98.0,1.0,0.0,8.0
3,10002013,23581541,39060235,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,3.0
4,10002155,23822395,33685454,0.0,0.0,1.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,3.0,3.0,0.0,2.0,0.0


In [ ]:
labs_high.nunique() # w count 

subject_id    44095
hadm_id       44095
stay_id       44095
hco3_high        62
bnp_high          6
bun_high        104
crp_high          9
esr_high          4
glu_high         22
hr_high         488
k_high           45
lac_high         94
map_high          1
na_high          92
rr_high         833
temp_high        89
tropT_high       21
wbc_high         85
dtype: int64

In [ ]:
labs_high.describe()

,subject_id,hadm_id,stay_id,hco3_high,bnp_high,bun_high,crp_high,esr_high,glu_high,hr_high,k_high,lac_high,map_high,na_high,rr_high,temp_high,tropT_high,wbc_high
count,4.409500e+04,4.409500e+04,4.409500e+04,8951.000000,1649.000000,23466.000000,1146.000000,365.000000,3620.000000,28641.000000,13694.000000,16285.000000,1.0,23515.000000,43131.000000,7733.000000,3152.000000,25463.000000
mean,1.499474e+07,2.498366e+07,3.497738e+07,4.303988,1.123711,6.818376,1.260035,1.106849,2.138122,32.740407,3.013290,4.501136,1.0,5.385966,52.905010,6.066339,3.107234,4.495464
std,2.890070e+06,2.885041e+06,2.897011e+06,6.097218,0.430034,9.377324,0.745589,0.373699,2.050411,62.378103,3.661173,7.427779,NaN,8.197488,120.970933,10.097283,2.174968,6.348174
min,1.000122e+07,2.000015e+07,3.000015e+07,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.248926e+07,2.249742e+07,3.244898e+07,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,4.000000,1.000000,1.000000,1.0,1.000000,8.000000,1.000000,2.000000,1.000000
50%,1.499689e+07,2.496239e+07,3.495532e+07,2.000000,1.000000,4.000000,1.000000,1.000000,1.000000,13.000000,2.000000,2.000000,1.0,2.000000,20.000000,3.000000,3.000000,3.000000
75%,1.750961e+07,2.747581e+07,3.748589e+07,5.000000,1.000000,8.000000,1.000000,1.000000,2.000000,36.000000,3.000000,5.000000,1.0,6.000000,48.000000,6.000000,4.000000,5.000000
max,1.999999e+07,2.999983e+07,3.999981e+07,97.000000,6.000000,148.000000,11.000000,4.000000,25.000000,1767.000000,70.000000,138.000000,1.0,168.000000,8505.000000,216.000000,24.000000,118.000000


In [ ]:
high_low = pd.merge(labs_high, labs_low, how = 'outer', on = ['subject_id', 'hadm_id', 'stay_id'])

print(high_low.shape)
high_low.head()

(44209, 28)


,subject_id,hadm_id,stay_id,hco3_high,bnp_high,bun_high,crp_high,esr_high,glu_high,hr_high,k_high,lac_high,map_high,na_high,rr_high,temp_high,tropT_high,wbc_high,hco3_low,hgb_low,hr_low,k_low,map_low,na_low,plt_low,rr_low,temp_low,wbc_low
0,10001217,24597018,37067082,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001725,25563031,31205490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001884,26184834,37510196,14.0,0.0,11.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,98.0,1.0,0.0,8.0,0.0,6.0,8.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0
3,10002013,23581541,39060235,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,10002155,23822395,33685454,0.0,0.0,1.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,3.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2.0


In [ ]:
labs_max = pd.read_csv('labs_max.csv', header = 0)
print(labs_max.shape)
labs_max.head()

(1113449, 5)


,subject_id,hadm_id,stay_id,feature,max_val
0,19452423,28656889,33252269,rr,26.0
1,19452423,28656889,33252269,spo2,100.0
2,19452423,28656889,33252269,hr,133.0
3,19452423,28656889,33252269,gcs_verbal,4.0
4,19452423,28656889,33252269,fio2,100.0


In [ ]:
test_max = labs_max.pivot_table('max_val', ['subject_id', 'hadm_id', 'stay_id'], 'feature')

test_max.reset_index(drop = False, inplace = True)
test_max = test_max.rename_axis(None, axis=1)
test_max.drop(test_max.columns[[19, 20, 21]], axis = 1, inplace = True)
test_max.columns = ['subject_id', 'hadm_id', 'stay_id', 'alt_max', 'ast_max', 'alkphos_max', 'ammonia_max', 'anion_gap_max', 'bicarb_max', 'bnp_max', 'bun_max', 'cl_max', 'ck_max', 'cortisol_max', 'cr_max', 'crp_max', 'dbili_max', 'esr_max', 'fio2_max', 'glu_max', 'hgb_max', 'hr_max', 'ht', 'inr_max', 'k_max', 'lac_max', 'lipase_max', 'map_max', 'na_max', 'pH_max', 'paco2_max', 'pao2_max', 'plt_max', 'rr_max', 'sao2_max', 'spo2_max', 'tbili_max', 'temp_max', 'transfusion_consent', 'tropT_max', 'wbc_max', 'wgt']
print(test_max.shape)
test_max.head()


(44245, 42)


,subject_id,hadm_id,stay_id,alt_max,ast_max,alkphos_max,ammonia_max,anion_gap_max,bicarb_max,bnp_max,bun_max,cl_max,ck_max,cortisol_max,cr_max,crp_max,dbili_max,esr_max,fio2_max,glu_max,hgb_max,hr_max,ht,inr_max,k_max,lac_max,lipase_max,map_max,na_max,pH_max,paco2_max,pao2_max,plt_max,rr_max,sao2_max,spo2_max,tbili_max,temp_max,transfusion_consent,tropT_max,wbc_max,wgt
0,10001217,24597018,37067082,NaN,NaN,NaN,NaN,15.0,23.0,NaN,9.0,104.0,NaN,NaN,0.4,NaN,NaN,NaN,NaN,113.0,11.2,106.0,NaN,1.2,3.6,NaN,NaN,NaN,138.0,NaN,NaN,NaN,285.0,27.0,NaN,99.0,NaN,100.8,NaN,NaN,19.0,NaN
1,10001725,25563031,31205490,NaN,NaN,NaN,NaN,16.0,24.0,NaN,18.0,106.0,NaN,NaN,0.8,NaN,NaN,NaN,NaN,152.0,13.9,91.0,NaN,1.0,3.9,NaN,NaN,NaN,140.0,NaN,NaN,NaN,330.0,23.0,NaN,100.0,NaN,98.4,NaN,NaN,20.1,NaN
2,10001884,26184834,37510196,823.0,474.0,71.0,NaN,15.0,37.0,NaN,38.0,100.0,786.0,NaN,1.3,NaN,NaN,NaN,100.0,203.0,12.1,87.0,NaN,1.5,7.1,1.5,NaN,NaN,140.0,7.46,60.0,91.0,196.0,32.0,92.0,100.0,0.5,101.0,NaN,0.08,18.4,NaN
3,10002013,23581541,39060235,NaN,NaN,NaN,NaN,12.0,23.0,NaN,16.0,109.0,NaN,NaN,1.1,NaN,NaN,NaN,100.0,216.0,12.0,105.0,NaN,1.2,4.7,3.3,NaN,NaN,140.0,7.40,56.0,462.0,254.0,23.0,99.0,100.0,NaN,97.8,NaN,NaN,20.2,NaN
4,10002155,23822395,33685454,NaN,NaN,NaN,NaN,14.0,29.0,NaN,21.0,107.0,589.0,NaN,1.2,NaN,NaN,NaN,95.0,116.0,12.5,108.0,NaN,3.0,4.7,NaN,NaN,NaN,143.0,7.43,48.0,76.0,185.0,22.0,94.0,99.0,NaN,98.8,NaN,3.99,6.5,53.0


In [ ]:
labs_min = pd.read_csv('labs_min.csv', header = 0)
print(labs_min.shape)
labs_min.head()

(1113449, 5)


,subject_id,hadm_id,stay_id,feature,min_val
0,11155072,22807589,32764467,spo2,95.0
1,11155072,22807589,32764467,hr,75.0
2,11155072,22807589,32764467,temp,97.6
3,11155072,22807589,32764467,bun,9.0
4,11155072,22807589,32764467,gcs_eye,4.0


In [ ]:
test_min = labs_min.pivot_table('min_val', ['subject_id', 'hadm_id', 'stay_id'], 'feature')

test_min.reset_index(drop = False, inplace = True)
test_min = test_min.rename_axis(None, axis=1)
test_min.drop(test_min.columns[[19, 20, 21]], axis = 1, inplace = True)
test_min.columns = ['subject_id', 'hadm_id', 'stay_id', 'alt_min', 'ast_min', 'alkphos_min', 'ammonia_min', 'anion_gap_min', 'bicarb_min', 'bnp_min', 'bun_min', 'cl_min', 'ck_min', 'cortisol_min', 'cr_min', 'crp_min', 'dbili_min', 'esr_min', 'fio2_min', 'glu_min', 'hgb_min', 'hr_min', 'ht', 'inr_min', 'k_min', 'lac_min', 'lipase_min', 'map_min', 'na_min', 'pH_min', 'paco2_min', 'pao2_min', 'plt_min', 'rr_min', 'sao2_min', 'spo2_min', 'tbili_min', 'temp_min', 'transfusion_consent', 'tropT_min', 'wbc_min', 'wgt']
print(test_min.shape)
test_min.head()

(44245, 42)


,subject_id,hadm_id,stay_id,alt_min,ast_min,alkphos_min,ammonia_min,anion_gap_min,bicarb_min,bnp_min,bun_min,cl_min,ck_min,cortisol_min,cr_min,crp_min,dbili_min,esr_min,fio2_min,glu_min,hgb_min,hr_min,ht,inr_min,k_min,lac_min,lipase_min,map_min,na_min,pH_min,paco2_min,pao2_min,plt_min,rr_min,sao2_min,spo2_min,tbili_min,temp_min,transfusion_consent,tropT_min,wbc_min,wgt
0,10001217,24597018,37067082,NaN,NaN,NaN,NaN,15.0,23.0,NaN,9.0,104.0,NaN,NaN,0.4,NaN,NaN,NaN,NaN,113.0,11.2,78.0,NaN,1.2,3.6,NaN,NaN,NaN,138.0,NaN,NaN,NaN,285.0,13.0,NaN,92.0,NaN,98.1,NaN,NaN,19.0,NaN
1,10001725,25563031,31205490,NaN,NaN,NaN,NaN,14.0,24.0,NaN,16.0,102.0,NaN,NaN,0.8,NaN,NaN,NaN,NaN,146.0,12.6,55.0,NaN,1.0,3.5,NaN,NaN,NaN,138.0,NaN,NaN,NaN,299.0,13.0,NaN,94.0,NaN,97.5,NaN,NaN,17.0,NaN
2,10001884,26184834,37510196,116.0,55.0,46.0,NaN,6.0,31.0,NaN,14.0,90.0,361.0,NaN,0.5,NaN,NaN,NaN,35.0,94.0,7.0,38.0,NaN,1.0,3.9,1.1,NaN,NaN,132.0,7.38,49.0,65.0,103.0,0.0,90.0,45.0,0.2,97.8,NaN,0.08,10.7,NaN
3,10002013,23581541,39060235,NaN,NaN,NaN,NaN,12.0,23.0,NaN,13.0,103.0,NaN,NaN,0.9,NaN,NaN,NaN,50.0,98.0,10.2,80.0,NaN,1.0,3.2,2.6,NaN,NaN,136.0,7.27,41.0,90.0,248.0,0.0,96.0,91.0,NaN,97.2,NaN,NaN,17.9,NaN
4,10002155,23822395,33685454,NaN,NaN,NaN,NaN,11.0,23.0,NaN,11.0,103.0,91.0,NaN,0.9,NaN,NaN,NaN,35.0,95.0,9.8,61.0,NaN,1.0,4.2,NaN,NaN,NaN,137.0,7.36,43.0,45.0,136.0,10.0,82.0,81.0,NaN,95.9,NaN,2.65,3.8,53.0


In [ ]:
min_max = pd.merge(test_min, test_max, how = "outer", on = ['subject_id', 'hadm_id', 'stay_id'])
print(min_max.shape)
min_max.head()

(44245, 81)


,subject_id,hadm_id,stay_id,alt_min,ast_min,alkphos_min,ammonia_min,anion_gap_min,bicarb_min,bnp_min,bun_min,cl_min,ck_min,cortisol_min,cr_min,crp_min,dbili_min,esr_min,fio2_min,glu_min,hgb_min,hr_min,ht_x,inr_min,k_min,lac_min,lipase_min,map_min,na_min,pH_min,paco2_min,pao2_min,plt_min,rr_min,sao2_min,spo2_min,tbili_min,temp_min,transfusion_consent_x,tropT_min,...,wgt_x,alt_max,ast_max,alkphos_max,ammonia_max,anion_gap_max,bicarb_max,bnp_max,bun_max,cl_max,ck_max,cortisol_max,cr_max,crp_max,dbili_max,esr_max,fio2_max,glu_max,hgb_max,hr_max,ht_y,inr_max,k_max,lac_max,lipase_max,map_max,na_max,pH_max,paco2_max,pao2_max,plt_max,rr_max,sao2_max,spo2_max,tbili_max,temp_max,transfusion_consent_y,tropT_max,wbc_max,wgt_y
0,10001217,24597018,37067082,NaN,NaN,NaN,NaN,15.0,23.0,NaN,9.0,104.0,NaN,NaN,0.4,NaN,NaN,NaN,NaN,113.0,11.2,78.0,NaN,1.2,3.6,NaN,NaN,NaN,138.0,NaN,NaN,NaN,285.0,13.0,NaN,92.0,NaN,98.1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,15.0,23.0,NaN,9.0,104.0,NaN,NaN,0.4,NaN,NaN,NaN,NaN,113.0,11.2,106.0,NaN,1.2,3.6,NaN,NaN,NaN,138.0,NaN,NaN,NaN,285.0,27.0,NaN,99.0,NaN,100.8,NaN,NaN,19.0,NaN
1,10001725,25563031,31205490,NaN,NaN,NaN,NaN,14.0,24.0,NaN,16.0,102.0,NaN,NaN,0.8,NaN,NaN,NaN,NaN,146.0,12.6,55.0,NaN,1.0,3.5,NaN,NaN,NaN,138.0,NaN,NaN,NaN,299.0,13.0,NaN,94.0,NaN,97.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,16.0,24.0,NaN,18.0,106.0,NaN,NaN,0.8,NaN,NaN,NaN,NaN,152.0,13.9,91.0,NaN,1.0,3.9,NaN,NaN,NaN,140.0,NaN,NaN,NaN,330.0,23.0,NaN,100.0,NaN,98.4,NaN,NaN,20.1,NaN
2,10001884,26184834,37510196,116.0,55.0,46.0,NaN,6.0,31.0,NaN,14.0,90.0,361.0,NaN,0.5,NaN,NaN,NaN,35.0,94.0,7.0,38.0,NaN,1.0,3.9,1.1,NaN,NaN,132.0,7.38,49.0,65.0,103.0,0.0,90.0,45.0,0.2,97.8,NaN,0.08,...,NaN,823.0,474.0,71.0,NaN,15.0,37.0,NaN,38.0,100.0,786.0,NaN,1.3,NaN,NaN,NaN,100.0,203.0,12.1,87.0,NaN,1.5,7.1,1.5,NaN,NaN,140.0,7.46,60.0,91.0,196.0,32.0,92.0,100.0,0.5,101.0,NaN,0.08,18.4,NaN
3,10002013,23581541,39060235,NaN,NaN,NaN,NaN,12.0,23.0,NaN,13.0,103.0,NaN,NaN,0.9,NaN,NaN,NaN,50.0,98.0,10.2,80.0,NaN,1.0,3.2,2.6,NaN,NaN,136.0,7.27,41.0,90.0,248.0,0.0,96.0,91.0,NaN,97.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12.0,23.0,NaN,16.0,109.0,NaN,NaN,1.1,NaN,NaN,NaN,100.0,216.0,12.0,105.0,NaN,1.2,4.7,3.3,NaN,NaN,140.0,7.40,56.0,462.0,254.0,23.0,99.0,100.0,NaN,97.8,NaN,NaN,20.2,NaN
4,10002155,23822395,33685454,NaN,NaN,NaN,NaN,11.0,23.0,NaN,11.0,103.0,91.0,NaN,0.9,NaN,NaN,NaN,35.0,95.0,9.8,61.0,NaN,1.0,4.2,NaN,NaN,NaN,137.0,7.36,43.0,45.0,136.0,10.0,82.0,81.0,NaN,95.9,NaN,2.65,...,53.0,NaN,NaN,NaN,NaN,14.0,29.0,NaN,21.0,107.0,589.0,NaN,1.2,NaN,NaN,NaN,95.0,116.0,12.5,108.0,NaN,3.0,4.7,NaN,NaN,NaN,143.0,7.43,48.0,76.0,185.0,22.0,94.0,99.0,NaN,98.8,NaN,3.99,6.5,53.0


In [ ]:
labs_last = pd.read_csv('labs_last.csv', header=0)
labs_last = labs_last.pivot_table('valuenum', ['subject_id', 'hadm_id', 'stay_id'], 'feature')
labs_last.reset_index(drop = False, inplace = True)
labs_last = labs_last.rename_axis(None, axis=1)
labs_last.drop(labs_last.columns[[19, 20, 21]], axis = 1, inplace = True)
labs_last.columns = ['subject_id', 'hadm_id', 'stay_id', 'alt_last', 'ast_last', 'alkphos_last', 'ammonia_last', 'anion_gap_last', 'hco3_last', 'bnp_last', 'bun_last', 'cl_last', 'ck_last', 'cortisol_last', 'cr_last', 'crp_last', 'dbili_last', 'esr_last', 'fio2_last', 'glu_last', 'hgb_last', 'hr_last', 'ht', 'inr_last', 'k_last', 'lac_last', 'lipase_last', 'map_last', 'na_last', 'pH_last', 'paco2_last', 'pao2_last', 'plt_last', 'rr_last', 'sao2_last', 'spo2_last', 'tbili_last', 'temp_last', 'transfusion_consent', 'tropT_last', 'wbc_last', 'wgt']
print(labs_last.shape)
labs_last.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(44245, 42)


,subject_id,hadm_id,stay_id,alt_last,ast_last,alkphos_last,ammonia_last,anion_gap_last,hco3_last,bnp_last,bun_last,cl_last,ck_last,cortisol_last,cr_last,crp_last,dbili_last,esr_last,fio2_last,glu_last,hgb_last,hr_last,ht,inr_last,k_last,lac_last,lipase_last,map_last,na_last,pH_last,paco2_last,pao2_last,plt_last,rr_last,sao2_last,spo2_last,tbili_last,temp_last,transfusion_consent,tropT_last,wbc_last,wgt
0,10001217,24597018,37067082,NaN,NaN,NaN,NaN,15.0,23.0,NaN,9.0,104.0,NaN,NaN,0.4,NaN,NaN,NaN,NaN,113.0,11.2,86.0,NaN,1.2,3.6,NaN,NaN,NaN,138.0,NaN,NaN,NaN,285.0,18.000000,NaN,99.0,NaN,98.5,NaN,NaN,19.0,NaN
1,10001725,25563031,31205490,NaN,NaN,NaN,NaN,14.0,24.0,NaN,16.0,106.0,NaN,NaN,0.8,NaN,NaN,NaN,NaN,146.0,13.9,55.0,NaN,1.0,3.9,NaN,NaN,NaN,140.0,NaN,NaN,NaN,330.0,19.000000,NaN,100.0,NaN,97.7,NaN,NaN,17.0,NaN
2,10001884,26184834,37510196,167.0,134.0,55.0,NaN,11.0,33.0,NaN,30.0,96.0,786.0,NaN,1.1,NaN,NaN,NaN,50.0,148.0,11.4,38.0,NaN,1.4,4.0,1.5,NaN,NaN,136.0,7.38,60.0,65.0,171.0,10.000000,90.0,98.0,0.4,98.1,NaN,0.08,18.4,NaN
3,10002013,23581541,39060235,NaN,NaN,NaN,NaN,12.0,23.0,NaN,16.0,104.0,NaN,NaN,1.1,NaN,NaN,NaN,100.0,216.0,11.9,80.0,NaN,1.2,3.4,3.3,NaN,NaN,137.0,7.35,45.0,421.0,252.0,9.333333,99.0,100.0,NaN,97.2,NaN,NaN,18.2,NaN
4,10002155,23822395,33685454,NaN,NaN,NaN,NaN,13.0,25.0,NaN,19.0,106.0,589.0,NaN,0.9,NaN,NaN,NaN,95.0,95.0,12.5,68.0,NaN,1.0,4.5,NaN,NaN,NaN,139.0,7.36,48.0,76.0,185.0,18.000000,94.0,97.0,NaN,95.9,NaN,3.99,5.5,53.0


In [ ]:
min_max_last = pd.merge(min_max, labs_last, how = "outer", on = ['subject_id', 'hadm_id', 'stay_id'])
print(min_max_last.shape)
min_max_last.head()

(44245, 120)


,subject_id,hadm_id,stay_id,alt_min,ast_min,alkphos_min,ammonia_min,anion_gap_min,bicarb_min,bnp_min,bun_min,cl_min,ck_min,cortisol_min,cr_min,crp_min,dbili_min,esr_min,fio2_min,glu_min,hgb_min,hr_min,ht_x,inr_min,k_min,lac_min,lipase_min,map_min,na_min,pH_min,paco2_min,pao2_min,plt_min,rr_min,sao2_min,spo2_min,tbili_min,temp_min,transfusion_consent_x,tropT_min,...,wgt_y,alt_last,ast_last,alkphos_last,ammonia_last,anion_gap_last,hco3_last,bnp_last,bun_last,cl_last,ck_last,cortisol_last,cr_last,crp_last,dbili_last,esr_last,fio2_last,glu_last,hgb_last,hr_last,ht,inr_last,k_last,lac_last,lipase_last,map_last,na_last,pH_last,paco2_last,pao2_last,plt_last,rr_last,sao2_last,spo2_last,tbili_last,temp_last,transfusion_consent,tropT_last,wbc_last,wgt
0,10001217,24597018,37067082,NaN,NaN,NaN,NaN,15.0,23.0,NaN,9.0,104.0,NaN,NaN,0.4,NaN,NaN,NaN,NaN,113.0,11.2,78.0,NaN,1.2,3.6,NaN,NaN,NaN,138.0,NaN,NaN,NaN,285.0,13.0,NaN,92.0,NaN,98.1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,15.0,23.0,NaN,9.0,104.0,NaN,NaN,0.4,NaN,NaN,NaN,NaN,113.0,11.2,86.0,NaN,1.2,3.6,NaN,NaN,NaN,138.0,NaN,NaN,NaN,285.0,18.000000,NaN,99.0,NaN,98.5,NaN,NaN,19.0,NaN
1,10001725,25563031,31205490,NaN,NaN,NaN,NaN,14.0,24.0,NaN,16.0,102.0,NaN,NaN,0.8,NaN,NaN,NaN,NaN,146.0,12.6,55.0,NaN,1.0,3.5,NaN,NaN,NaN,138.0,NaN,NaN,NaN,299.0,13.0,NaN,94.0,NaN,97.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,14.0,24.0,NaN,16.0,106.0,NaN,NaN,0.8,NaN,NaN,NaN,NaN,146.0,13.9,55.0,NaN,1.0,3.9,NaN,NaN,NaN,140.0,NaN,NaN,NaN,330.0,19.000000,NaN,100.0,NaN,97.7,NaN,NaN,17.0,NaN
2,10001884,26184834,37510196,116.0,55.0,46.0,NaN,6.0,31.0,NaN,14.0,90.0,361.0,NaN,0.5,NaN,NaN,NaN,35.0,94.0,7.0,38.0,NaN,1.0,3.9,1.1,NaN,NaN,132.0,7.38,49.0,65.0,103.0,0.0,90.0,45.0,0.2,97.8,NaN,0.08,...,NaN,167.0,134.0,55.0,NaN,11.0,33.0,NaN,30.0,96.0,786.0,NaN,1.1,NaN,NaN,NaN,50.0,148.0,11.4,38.0,NaN,1.4,4.0,1.5,NaN,NaN,136.0,7.38,60.0,65.0,171.0,10.000000,90.0,98.0,0.4,98.1,NaN,0.08,18.4,NaN
3,10002013,23581541,39060235,NaN,NaN,NaN,NaN,12.0,23.0,NaN,13.0,103.0,NaN,NaN,0.9,NaN,NaN,NaN,50.0,98.0,10.2,80.0,NaN,1.0,3.2,2.6,NaN,NaN,136.0,7.27,41.0,90.0,248.0,0.0,96.0,91.0,NaN,97.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12.0,23.0,NaN,16.0,104.0,NaN,NaN,1.1,NaN,NaN,NaN,100.0,216.0,11.9,80.0,NaN,1.2,3.4,3.3,NaN,NaN,137.0,7.35,45.0,421.0,252.0,9.333333,99.0,100.0,NaN,97.2,NaN,NaN,18.2,NaN
4,10002155,23822395,33685454,NaN,NaN,NaN,NaN,11.0,23.0,NaN,11.0,103.0,91.0,NaN,0.9,NaN,NaN,NaN,35.0,95.0,9.8,61.0,NaN,1.0,4.2,NaN,NaN,NaN,137.0,7.36,43.0,45.0,136.0,10.0,82.0,81.0,NaN,95.9,NaN,2.65,...,53.0,NaN,NaN,NaN,NaN,13.0,25.0,NaN,19.0,106.0,589.0,NaN,0.9,NaN,NaN,NaN,95.0,95.0,12.5,68.0,NaN,1.0,4.5,NaN,NaN,NaN,139.0,7.36,48.0,76.0,185.0,18.000000,94.0,97.0,NaN,95.9,NaN,3.99,5.5,53.0


In [ ]:
labs = pd.merge(min_max_last, high_low, how = 'outer', on = ['subject_id', 'hadm_id', 'stay_id'])
print(labs.shape)
labs.head()

(44245, 145)


,subject_id,hadm_id,stay_id,alt_min,ast_min,alkphos_min,ammonia_min,anion_gap_min,bicarb_min,bnp_min,bun_min,cl_min,ck_min,cortisol_min,cr_min,crp_min,dbili_min,esr_min,fio2_min,glu_min,hgb_min,hr_min,ht_x,inr_min,k_min,lac_min,lipase_min,map_min,na_min,pH_min,paco2_min,pao2_min,plt_min,rr_min,sao2_min,spo2_min,tbili_min,temp_min,transfusion_consent_x,tropT_min,...,map_last,na_last,pH_last,paco2_last,pao2_last,plt_last,rr_last,sao2_last,spo2_last,tbili_last,temp_last,transfusion_consent,tropT_last,wbc_last,wgt,hco3_high,bnp_high,bun_high,crp_high,esr_high,glu_high,hr_high,k_high,lac_high,map_high,na_high,rr_high,temp_high,tropT_high,wbc_high,hco3_low,hgb_low,hr_low,k_low,map_low,na_low,plt_low,rr_low,temp_low,wbc_low
0,10001217,24597018,37067082,NaN,NaN,NaN,NaN,15.0,23.0,NaN,9.0,104.0,NaN,NaN,0.4,NaN,NaN,NaN,NaN,113.0,11.2,78.0,NaN,1.2,3.6,NaN,NaN,NaN,138.0,NaN,NaN,NaN,285.0,13.0,NaN,92.0,NaN,98.1,NaN,NaN,...,NaN,138.0,NaN,NaN,NaN,285.0,18.000000,NaN,99.0,NaN,98.5,NaN,NaN,19.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001725,25563031,31205490,NaN,NaN,NaN,NaN,14.0,24.0,NaN,16.0,102.0,NaN,NaN,0.8,NaN,NaN,NaN,NaN,146.0,12.6,55.0,NaN,1.0,3.5,NaN,NaN,NaN,138.0,NaN,NaN,NaN,299.0,13.0,NaN,94.0,NaN,97.5,NaN,NaN,...,NaN,140.0,NaN,NaN,NaN,330.0,19.000000,NaN,100.0,NaN,97.7,NaN,NaN,17.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001884,26184834,37510196,116.0,55.0,46.0,NaN,6.0,31.0,NaN,14.0,90.0,361.0,NaN,0.5,NaN,NaN,NaN,35.0,94.0,7.0,38.0,NaN,1.0,3.9,1.1,NaN,NaN,132.0,7.38,49.0,65.0,103.0,0.0,90.0,45.0,0.2,97.8,NaN,0.08,...,NaN,136.0,7.38,60.0,65.0,171.0,10.000000,90.0,98.0,0.4,98.1,NaN,0.08,18.4,NaN,14.0,0.0,11.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,98.0,1.0,0.0,8.0,0.0,6.0,8.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0
3,10002013,23581541,39060235,NaN,NaN,NaN,NaN,12.0,23.0,NaN,13.0,103.0,NaN,NaN,0.9,NaN,NaN,NaN,50.0,98.0,10.2,80.0,NaN,1.0,3.2,2.6,NaN,NaN,136.0,7.27,41.0,90.0,248.0,0.0,96.0,91.0,NaN,97.2,NaN,NaN,...,NaN,137.0,7.35,45.0,421.0,252.0,9.333333,99.0,100.0,NaN,97.2,NaN,NaN,18.2,NaN,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,10002155,23822395,33685454,NaN,NaN,NaN,NaN,11.0,23.0,NaN,11.0,103.0,91.0,NaN,0.9,NaN,NaN,NaN,35.0,95.0,9.8,61.0,NaN,1.0,4.2,NaN,NaN,NaN,137.0,7.36,43.0,45.0,136.0,10.0,82.0,81.0,NaN,95.9,NaN,2.65,...,NaN,139.0,7.36,48.0,76.0,185.0,18.000000,94.0,97.0,NaN,95.9,NaN,3.99,5.5,53.0,0.0,0.0,1.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,3.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2.0


In [ ]:
pao2 = pd.read_csv('pao2.csv', header = 0)
print(pao2.shape)
pao2.head()

In [ ]:
fio2 = pd.read_csv('fio2.csv', header = 0)
print(fio2.shape)
fio2.head()

In [ ]:
vent = pd.read_csv('vent.csv', header = 0)
print(vent.shape)
vent.head()

# Define Cohort.

## (RUN ONCE) Get intubated subjects and intubation variables.

In [ ]:
### Cycle through chartevents.

chartevents = pd.read_csv('/content/drive/MyDrive/BMI212/chartevents/CHARTEVENTS_0.csv', header = 0)
print(chartevents.shape)
chartevents.head()

In [ ]:
# Pull patients who were intubated.
vent_type_id = [722,223849]
arf_mechvent = chartevents.loc[np.where(chartevents['ITEMID'].isin(vent_type_id) == True)[0]] 
len(arf_mechvent['SUBJECT_ID'].unique())


In [ ]:
# See counts of ventilator types.
arf_mechvent['VALUE'].value_counts()

Series([], Name: VALUE, dtype: int64)

In [ ]:
# Remove ventilator types for CPAP.
arf_mechvent = arf_mechvent[-arf_mechvent['VALUE'].str.contains('CPAP', na=False)]
arf_mechvent['VALUE'].value_counts()

In [ ]:
### Write results to csv file.
# name of csv file 
filename = "mechvent_subj0.csv"
    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the header
    csvwriter.writerow(arf_mechvent.columns) 
        
    # writing the data rows 
    for i in range(arf_mechvent.shape[0]):
      csvwriter.writerow(arf_mechvent.iloc[i])

## Extracting ICD9 diagnoses: ARF, COPD.


In [ ]:
### ICD9 diagnoses linked to admission and subject ID.
diagnoses_icd = pd.read_csv('DIAGNOSES_ICD.csv', header = 0)
print(diagnoses_icd.shape)
diagnoses_icd.head()

(651047, 5)


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [ ]:
### ICD9 code diagnoses descriptions.
# Check acute respiratory failure diagnosis.
icd_dx = pd.read_csv('D_ICD_DIAGNOSES.csv', header = 0)
print(icd_dx.shape)

# ID search grids for acute respiratory failure, COPD.
arf_searchgrid = ['51881','51884']
bronchitis_searchgrid = ['4910','4911','49120','49121','49122','4918', '4919']
emphysema_searchgrid = ['4928', '5181','5182','7702']
copd_searchgrid = np.concatenate((bronchitis_searchgrid, emphysema_searchgrid))

icd_dx[icd_dx['ICD9_CODE'].str.contains('51884')]


(14567, 4)


,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
5553,5282,51884,Acute & chronc resp fail,Acute and chronic respiratory failure


In [ ]:
# Get COPD diagnoses.
copd_dx = diagnoses_icd.loc[np.where(diagnoses_icd['ICD9_CODE'].isin(copd_searchgrid) == True)[0]]
print('No. COPD total: ', len(copd_dx['SUBJECT_ID']))
print('No. COPD unique: ', len(copd_dx['SUBJECT_ID'].unique()))

copd_dx.drop(copd_dx.columns[[0,3,4]], axis = 1, inplace = True)
copd_dx.head()

No. COPD total:  2172
No. COPD unique:  1776


,SUBJECT_ID,HADM_ID
141,124,138376
841,33,176176
2391,165,170252
2776,111,192123
3843,480,182365


In [ ]:
# Get acute respiratory failure diagnoses.
arf_dx = diagnoses_icd.loc[np.where(diagnoses_icd['ICD9_CODE'].isin(arf_searchgrid) == True)[0]]
print('No. acute respiratory failure total: ', len(arf_dx['SUBJECT_ID']))
print('No. acute respiratory failure unique: ', len(arf_dx['SUBJECT_ID'].unique()))

#arf_dx = arf_dx[arf_dx['SEQ_NUM'] == 1.0]
#print('No. acute respiratory failure primary dx: ', arf_dx['SUBJECT_ID'].nunique())

arf_dx.drop(arf_dx.columns[[0,3,4]], axis = 1, inplace = True)
arf_dx.head()


No. acute respiratory failure total:  8181
No. acute respiratory failure unique:  7145


,SUBJECT_ID,HADM_ID
491,91,121205
518,94,140037
564,101,175533
1517,250,124271
1535,252,190159


## Extracting ICD9 procedures: intubation.

In [ ]:
### ICD9 code procedures descriptions.
icd_proc = pd.read_csv('D_ICD_PROCEDURES.csv', header = 0)
print(icd_proc.shape)
icd_proc.head()

(3882, 4)


,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,264,851,Canthotomy,Canthotomy
1,265,852,Blepharorrhaphy,Blepharorrhaphy
2,266,859,Adjust lid position NEC,Other adjustment of lid position
3,267,861,Lid reconst w skin graft,Reconstruction of eyelid with skin flap or graft
4,268,862,Lid reconst w muc graft,Reconstruction of eyelid with mucous membrane ...


In [ ]:
### Search for invasive mechanical ventilation ICD9 procedure codes.
icd_proc[icd_proc['LONG_TITLE'].str.contains('mechanical ventilation', na = False)]
icd_vent_searchgrid = ['9670','9671','9672'] #ICD9 for invasive mechanical ventilation.

In [ ]:
### Get procedure ICDs.
procedures = pd.read_csv('PROCEDURES_ICD.csv', header = 0)
print(procedures.shape)
procedures.head()

(240095, 5)


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331


In [ ]:
### Get patients with mechanical ventilation.
icd_vent_pts = procedures.loc[np.where(procedures['ICD9_CODE'].isin(icd_vent_searchgrid) == True)[0]] 
print('No. intubated patients: ', icd_vent_pts['SUBJECT_ID'].nunique())

icd_vent_pts.drop(icd_vent_pts.columns[[0,3,4]], axis = 1, inplace = True)
icd_vent_pts.head()

icd_vent_pts.head()


No. intubated patients:  13098


,SUBJECT_ID,HADM_ID
1,2592,130856
3,55357,119355
14,16994,115444
40,4454,177326
63,755,123796


## Mechanical ventilation indicators.

In [ ]:
### Get list of unique hospital admissions with mechanical ventilation.
arf1 = pd.read_csv('mechvent_subj1.csv', header = 0)
arf2 = pd.read_csv('mechvent_subj2.csv', header = 0)
arf3 = pd.read_csv('mechvent_subj3.csv', header = 0)
arf4 = pd.read_csv('mechvent_subj4.csv', header = 0)
arf5 = pd.read_csv('mechvent_subj5.csv', header = 0)
arf6 = pd.read_csv('mechvent_subj6.csv', header = 0)
arf7 = pd.read_csv('mechvent_subj7.csv', header = 0)
arf8 = pd.read_csv('mechvent_subj8.csv', header = 0)
arf9 = pd.read_csv('mechvent_subj9.csv', header = 0)
arf10 = pd.read_csv('mechvent_subj10.csv', header = 0)
arf11 = pd.read_csv('mechvent_subj11.csv', header = 0)
arf13 = pd.read_csv('mechvent_subj13.csv', header = 0)
arf14 = pd.read_csv('mechvent_subj14.csv', header = 0)
arf15 = pd.read_csv('mechvent_subj15.csv', header = 0)

#mechvent_pts = np.concatenate((arf1['HADM_ID'].unique(),arf2['HADM_ID'].unique(),
#                               arf3['HADM_ID'].unique(),arf4['HADM_ID'].unique(),
#                               arf5['HADM_ID'].unique(),arf6['HADM_ID'].unique(),
#                               arf7['HADM_ID'].unique(),arf8['HADM_ID'].unique(),
#                               arf9['HADM_ID'].unique(),arf10['HADM_ID'].unique(),
#                               arf11['HADM_ID'].unique(),arf13['HADM_ID'].unique(),
#                               arf14['HADM_ID'].unique(), arf15['HADM_ID'].unique()))

mechvent_pts = pd.concat([arf1, arf2, arf3, arf4, arf5, arf6, arf7, arf8, arf9, arf10, arf11, arf13, arf14, arf15])
mechvent_pts = mechvent_pts[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'VALUE']] #keep necessary cols
mechvent_pts = mechvent_pts[-(mechvent_pts['VALUE'].isna() == True)] #remove NA
print('No. intubated patients: ', mechvent_pts['SUBJECT_ID'].nunique())


No. intubated patients:  20192


## Extract intubated patients with ARF.

In [ ]:
### Load data.
admissions = pd.read_csv('ADMISSIONS.csv', header = 0, delimiter = ',')

del admissions['ROW_ID']
admissions = admissions.reset_index()
admissions.head()
del admissions['index']

print(admissions.shape)
admissions.head()

(58976, 18)


,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [ ]:
### ICU stays.
icustay = pd.read_csv('ICUSTAYS.csv', header = 0)

del icustay['ROW_ID']
icustay = icustay.reset_index()
del icustay['index']

print(icustay.shape)
icustay.head()

(61532, 11)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [ ]:
### COHORT DEFINITOIN
# Cohort: Mechnically ventilated patients with ARF dx. 
#arf_vent1 = pd.merge(arf_dx, mechvent_pts, on = ['SUBJECT_ID', 'HADM_ID'])
#arf_vent2 = pd.merge(arf_dx, icd_vent_pts, on = ['SUBJECT_ID', 'HADM_ID'])
#cohort_prelim = pd.merge(arf_vent1, arf_vent2, how = 'outer', on = ['SUBJECT_ID', 'HADM_ID'])
cohort_prelim = pd.merge(arf_dx, icd_vent_pts, on = ['SUBJECT_ID', 'HADM_ID'])
print('No. mechnically ventilated w ARF patients: ', cohort_prelim['SUBJECT_ID'].nunique())

cohort_prelim.head()

No. mechnically ventilated w ARF patients:  5825


,SUBJECT_ID,HADM_ID
0,91,121205
1,94,140037
2,101,175533
3,250,124271
4,252,190159


In [ ]:
### Cohort: add icu stays
cohort_prelim = pd.merge(cohort_prelim, icustay, on = ['SUBJECT_ID', 'HADM_ID'])
print('No. icu stays w arf and intubation: ', len(cohort_prelim['ICUSTAY_ID'].unique()))
print('No. patients w arf and intubation: ', len(cohort_prelim['SUBJECT_ID'].unique()))

### Cohort: length of icu stay > 1d
cohort_prelim = cohort_prelim[cohort_prelim['LOS'] > 1]
print('No. icu stays > 24h w intubation: ', len(cohort_prelim['ICUSTAY_ID'].unique()))
print('No. patients > 24h icu stay w intubation: ', len(cohort_prelim['SUBJECT_ID'].unique()))

cohort_prelim.head()


No. icu stays w arf and intubation:  7436
No. patients w arf and intubation:  5821
No. icu stays > 24h w intubation:  7042
No. patients > 24h icu stay w intubation:  5586


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
1,91,121205,256972,carevue,MICU,MICU,52,52,2177-05-07 03:52:00,2177-05-10 15:16:00,3.4750
2,94,140037,213450,carevue,MICU,MICU,52,52,2176-09-02 16:27:46,2176-09-25 14:00:30,22.8977
3,101,175533,233111,carevue,MICU,MICU,52,52,2196-09-26 18:37:40,2196-10-06 16:01:56,9.8919
4,250,124271,205951,carevue,MICU,CSRU,15,14,2188-11-12 09:25:47,2188-11-23 01:24:45,10.6659
5,252,190159,221239,carevue,MICU,MICU,12,12,2133-03-31 04:24:30,2133-04-10 20:52:09,10.6859


In [ ]:
### Get patient age
patients = pd.read_csv('PATIENTS.csv', header=0)
patients.head()
del patients['ROW_ID']
patients = patients.reset_index()
del patients['index']

pt_info = pd.merge(admissions, patients, how = 'left', on = ['SUBJECT_ID'])
#pt_info['age'] = pt_info['ADMITTIME'] - pt_info['DOB']
pt_info.loc[0]['ADMITTIME']
from datetime import datetime
age = []
for i in range(pt_info.shape[0]):
  admit = datetime.strptime(pt_info.loc[i]['ADMITTIME'],'%Y-%m-%d %H:%M:%S')
  admit = admit.year
  dob = datetime.strptime(pt_info.loc[i]['DOB'],'%Y-%m-%d %H:%M:%S')
  dob = dob.year
  age.append(admit-dob)

pt_info['age'] = age

In [ ]:
# Cohort: Remove < 18yo.
cohort_prelim = pd.merge(cohort_prelim, pt_info, on = ['SUBJECT_ID', 'HADM_ID'])
#cohort_final = cohort_final.sort_values('HADM_ID')
cohort_prelim[cohort_prelim['age'] >= 18]

print('No. icu stays: ', cohort_prelim['ICUSTAY_ID'].nunique())
print('No. patients: ', cohort_prelim['SUBJECT_ID'].nunique())

cohort_prelim.head()


No. icu stays:  7042
No. patients:  5586


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,age
0,91,121205,256972,carevue,MICU,MICU,52,52,2177-05-07 03:52:00,2177-05-10 15:16:00,3.4750,2177-04-23 00:08:00,2177-05-10 15:16:00,2177-05-10 15:16:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,JEWISH,MARRIED,WHITE,2177-04-22 21:02:00,2177-04-23 04:03:00,FEVER,1,1,F,2095-09-26 00:00:00,2177-05-10 00:00:00,2177-05-10 00:00:00,2177-05-10 00:00:00,1,82
1,94,140037,213450,carevue,MICU,MICU,52,52,2176-09-02 16:27:46,2176-09-25 14:00:30,22.8977,2176-09-02 14:22:00,2176-09-25 13:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,CANT,NOT SPECIFIED,MARRIED,ASIAN,2176-09-02 09:54:00,2176-09-02 17:56:00,PNEUMONIA,0,1,M,2101-09-20 00:00:00,2178-08-17 00:00:00,NaN,2178-08-17 00:00:00,1,75
2,101,175533,233111,carevue,MICU,MICU,52,52,2196-09-26 18:37:40,2196-10-06 16:01:56,9.8919,2196-09-26 18:36:00,2196-10-12 13:17:00,2196-10-12 13:17:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,NaN,MARRIED,ASIAN,2196-09-26 12:50:00,2196-09-26 18:37:00,RESPIRATORY FAILURE,1,1,M,2114-07-22 00:00:00,2196-10-12 00:00:00,2196-10-12 00:00:00,NaN,1,82
3,250,124271,205951,carevue,MICU,CSRU,15,14,2188-11-12 09:25:47,2188-11-23 01:24:45,10.6659,2188-11-12 09:22:00,2188-11-22 12:00:00,2188-11-22 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Self Pay,HAIT,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2188-11-12 06:56:00,2188-11-12 10:10:00,PNEUMONIA;R/O TB,1,1,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1,24
4,252,190159,221239,carevue,MICU,MICU,12,12,2133-03-31 04:24:30,2133-04-10 20:52:09,10.6859,2133-03-31 04:24:00,2133-04-23 15:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,NaN,UNOBTAINABLE,SINGLE,WHITE,2133-03-31 03:58:00,2133-03-31 05:00:00,GASTROINTESTINAL BLEED,0,1,M,2078-03-06 00:00:00,NaN,NaN,NaN,0,55


In [ ]:
### Cohort: Keep only first hospital admission.
cohort_prelim = cohort_prelim.sort_values('SUBJECT_ID')
cohort_prelim = cohort_prelim.reset_index()
del cohort_prelim['index']
print(cohort_prelim.shape[0], cohort_prelim['ICUSTAY_ID'].nunique(), cohort_prelim['HADM_ID'].nunique(), cohort_prelim['SUBJECT_ID'].nunique())

result = cohort_prelim.groupby('SUBJECT_ID')['ADMITTIME'].agg([min])
result.index.name = 'SUBJECT_ID'
result.reset_index(inplace=True)

print(result.shape[0], result['SUBJECT_ID'].nunique())

cohort_prelim_filter = pd.merge(cohort_prelim, result, on = ['SUBJECT_ID'])


cohort_prelim_filter = cohort_prelim_filter.sort_values(['SUBJECT_ID','HADM_ID','INTIME'])
cohort_prelim_filter = cohort_prelim_filter.reset_index()
del cohort_prelim_filter['index']

#df.sort_values(['a', 'b'], ascending=[True, False])
index = []
for i in range(cohort_prelim_filter.shape[0]):
  if cohort_prelim_filter.loc[i]['ADMITTIME'] == cohort_prelim_filter.loc[i]['min']:
    continue
  else: 
    index.append(i)

print(len(index))
cohort_prelim_filter = cohort_prelim_filter.drop(index)
print(cohort_prelim_filter.shape[0], cohort_prelim_filter['ICUSTAY_ID'].nunique(), cohort_prelim_filter['HADM_ID'].nunique(), cohort_prelim_filter['SUBJECT_ID'].nunique())
print(cohort_prelim.shape[0] - cohort_prelim_filter.shape[0])

7611 7042 6202 5586
5586 5586
746
6865 6351 5586 5586
746


In [ ]:
### Cohort split.
#cohort_readmit = cohort_prelim[cohort_prelim.duplicated(['SUBJECT_ID', 'HADM_ID'])]

e = cohort_prelim_filter['HADM_ID'].value_counts()
cohort_readmit = cohort_prelim_filter[cohort_prelim_filter['HADM_ID'].isin(e[e>1].index)]
cohort_readmit = cohort_readmit.sort_values(['SUBJECT_ID','HADM_ID'])
cohort_readmit = cohort_readmit.reset_index()
del cohort_readmit['index']
print(cohort_readmit.shape[0], cohort_readmit['ICUSTAY_ID'].nunique(), cohort_readmit['HADM_ID'].nunique(), cohort_readmit['SUBJECT_ID'].nunique())


cohort_nonreadmit = cohort_prelim_filter[cohort_prelim_filter['HADM_ID'].isin(e[e<=1].index)]
cohort_nonreadmit = cohort_nonreadmit.reset_index()
del cohort_nonreadmit['index']

print('Outcome cohort: ', cohort_readmit.shape[0], cohort_readmit['HADM_ID'].nunique(), len(cohort_readmit['SUBJECT_ID'].unique()))
print('Control cohort: ', cohort_nonreadmit.shape[0], cohort_nonreadmit['HADM_ID'].nunique(), len(cohort_nonreadmit['SUBJECT_ID'].unique()))

2149 1635 870 870
Outcome cohort:  2149 870 870
Control cohort:  4716 4716 4716


In [ ]:
# Split cohort: Nonreadmits -- remove discharge out directly from ICU.
cohort_nonreadmit_final = cohort_nonreadmit[-(cohort_nonreadmit['OUTTIME'] == cohort_nonreadmit['DISCHTIME'])]
print('No. patients: ', cohort_nonreadmit_final.shape[0], cohort_nonreadmit_final['ICUSTAY_ID'].nunique(), cohort_nonreadmit_final['HADM_ID'].nunique(), cohort_nonreadmit_final['SUBJECT_ID'].nunique())

No. patients:  4597 4597 4597 4597


In [ ]:
# Split cohort: get first icu stay, first icu readmission (if applicable).
# remove duplicate icu stays before removing >2 icu stays.

e = cohort_readmit['ICUSTAY_ID'].value_counts()
cohort_readmit_filter = cohort_readmit[cohort_readmit['ICUSTAY_ID'].isin(e[e <= 1].index)]
#cohort_readmit_filter = cohort_readmit_filter.sort_values(['SUBJECT_ID','HADM_ID'])
cohort_readmit_filter = cohort_readmit_filter.reset_index()
del cohort_readmit_filter['index']
print(cohort_readmit_filter.shape[0], cohort_readmit_filter['ICUSTAY_ID'].nunique(), cohort_readmit_filter['HADM_ID'].nunique(), cohort_readmit_filter['SUBJECT_ID'].nunique())

1167 1167 524 524


In [ ]:
# Split cohort: get first icu stay, first icu readmission.

cohort_readmit_filter_icu = cohort_readmit_filter.sort_values(['HADM_ID','INTIME'], ascending=[True, True])
cohort_readmit_filter_icu = cohort_readmit_filter_icu.reset_index()
del cohort_readmit_filter_icu['index']
cohort_readmit_filter_icu.head()

count = 1
drop = []
for i in range(cohort_readmit_filter_icu.shape[0]-1):
  if cohort_readmit_filter_icu.loc[i]['HADM_ID'] == cohort_readmit_filter_icu.loc[i+1]['HADM_ID']:
    count += 1
    if count >= 3:
      drop.append(i+1)
    else: 
      continue
  else:
    count = 1

print(cohort_readmit_filter_icu.shape[0], cohort_readmit_filter_icu['ICUSTAY_ID'].nunique(), 
      cohort_readmit_filter_icu['HADM_ID'].nunique(), cohort_readmit_filter_icu['SUBJECT_ID'].nunique())

cohort_readmit_filter_icu = cohort_readmit_filter_icu.drop(drop)
cohort_readmit_filter_icu = cohort_readmit_filter_icu.reset_index()
del cohort_readmit_filter_icu['index']
print(len(drop))

print(cohort_readmit_filter_icu.shape[0], cohort_readmit_filter_icu['ICUSTAY_ID'].nunique(), 
      cohort_readmit_filter_icu['HADM_ID'].nunique(), cohort_readmit_filter_icu['SUBJECT_ID'].nunique())

#cohort_readmit_filter_icu.loc[0:14][:]

#cohort_readmit_filter = cohort_readmit_filter.drop(index)


1167 1167 524 524
119
1048 1048 524 524


In [ ]:
# Split cohort: Readmission <= 7d
# take out both patient lines if icu readmission > 7d

drop = []
for i in range(cohort_readmit_filter_icu.shape[0]):
  if (i % 2) == 1: 
    time_dc = datetime.strptime(cohort_readmit_filter_icu.loc[i-1]['OUTTIME'],'%Y-%m-%d %H:%M:%S')
    time_dc = time_dc.day
    time_readmit = datetime.strptime(cohort_readmit_filter_icu.loc[i]['INTIME'],'%Y-%m-%d %H:%M:%S')
    time_readmit = time_readmit.day
    if time_readmit - time_dc > 7:
      drop.append(i-1)
      drop.append(i)
    else:
      continue

cohort_readmit_final = cohort_readmit_filter_icu.drop(drop)
print('No. dropped: ', len(drop))
print('Final readmit cohort: ', cohort_readmit_final.shape[0], cohort_readmit_final['ICUSTAY_ID'].nunique(), cohort_readmit_final['HADM_ID'].nunique(), cohort_readmit_final['SUBJECT_ID'].nunique())
cohort_readmit_final.head()
    


No. dropped:  156
Final readmit cohort:  892 892 446 446


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,age,min
0,49079,100234,256814,metavision,MICU,MICU,50,50,2118-10-23 19:35:15,2118-10-26 16:08:35,2.8565,2118-10-23 19:34:00,2118-11-05 05:36:00,2118-11-05 05:36:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,2118-10-23 17:04:00,2118-10-23 20:52:00,PNEUMONIA,1,1,M,2035-03-06 00:00:00,2118-11-05 00:00:00,2118-11-05 00:00:00,2118-11-05 00:00:00,1,83,2118-10-23 19:34:00
1,49079,100234,262878,metavision,CCU,MICU,7,50,2118-11-04 02:21:37,2118-11-05 08:36:13,1.2601,2118-10-23 19:34:00,2118-11-05 05:36:00,2118-11-05 05:36:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,2118-10-23 17:04:00,2118-10-23 20:52:00,PNEUMONIA,1,1,M,2035-03-06 00:00:00,2118-11-05 00:00:00,2118-11-05 00:00:00,2118-11-05 00:00:00,1,83,2118-10-23 19:34:00
2,5476,100343,207651,carevue,MICU,MICU,15,15,2132-04-20 09:20:36,2132-04-26 21:27:06,6.5045,2132-04-19 16:34:00,2132-05-19 13:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,JEWISH,WIDOWED,WHITE,2132-04-19 14:07:00,2132-04-19 18:02:00,RESPIRATORY DISTRESS,0,1,F,2057-05-11 00:00:00,2139-12-05 00:00:00,2139-12-05 00:00:00,2139-12-05 00:00:00,1,75,2132-04-19 16:34:00
3,5476,100343,244893,carevue,CSRU,CSRU,14,14,2132-04-30 08:31:14,2132-05-19 13:49:00,19.2207,2132-04-19 16:34:00,2132-05-19 13:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,JEWISH,WIDOWED,WHITE,2132-04-19 14:07:00,2132-04-19 18:02:00,RESPIRATORY DISTRESS,0,1,F,2057-05-11 00:00:00,2139-12-05 00:00:00,2139-12-05 00:00:00,2139-12-05 00:00:00,1,75,2132-04-19 16:34:00
4,80030,100442,249621,metavision,MICU,MICU,50,50,2119-06-07 02:44:09,2119-06-14 18:38:33,7.6628,2119-06-07 02:43:00,2119-07-18 17:15:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,LONG TERM CARE HOSPITAL,Private,ENGL,CATHOLIC,MARRIED,WHITE,2119-06-06 21:12:00,2119-06-07 03:25:00,HYPOTENSION,0,1,F,2063-07-15 00:00:00,NaN,NaN,NaN,0,56,2119-06-07 02:43:00


## Final cohort definition.

In [ ]:
### Combine cohort with outcome label.

cohort_readmit_final['outcome'] = np.ones((cohort_readmit_final.shape[0],1))
cohort_readmit_final = cohort_readmit_final.sort_values('HADM_ID')
cohort_readmit_final = cohort_readmit_final.reset_index()
del cohort_readmit_final['index']

cohort_readmit_icu1 = cohort_readmit_final.drop_duplicates(subset=['HADM_ID'], keep='first', inplace=False)
cohort_readmit_icu2 = cohort_readmit_final.drop_duplicates(subset=['HADM_ID'], keep='last', inplace=False)
#cohort_readmit_icu1 = cohort_readmit_final[(cohort_readmit_final.index % 2 != 1)]
#cohort_readmit_icu2 = cohort_readmit_final[(cohort_readmit_final.index % 2 == 1)]

cohort_nonreadmit_final['outcome'] = np.zeros((cohort_nonreadmit_final.shape[0],1))

cohort = pd.concat([cohort_readmit_icu1, cohort_nonreadmit_final],axis=0)
cohort = cohort.reset_index()
del cohort['index']
cohort = pd.DataFrame(cohort)
print(cohort.shape, cohort['SUBJECT_ID'].nunique())
cohort.head()

(5043, 36) 5043


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,age,min,outcome
0,49079,100234,256814,metavision,MICU,MICU,50,50,2118-10-23 19:35:15,2118-10-26 16:08:35,2.8565,2118-10-23 19:34:00,2118-11-05 05:36:00,2118-11-05 05:36:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,2118-10-23 17:04:00,2118-10-23 20:52:00,PNEUMONIA,1,1,M,2035-03-06 00:00:00,2118-11-05 00:00:00,2118-11-05 00:00:00,2118-11-05 00:00:00,1,83,2118-10-23 19:34:00,1.0
1,5476,100343,207651,carevue,MICU,MICU,15,15,2132-04-20 09:20:36,2132-04-26 21:27:06,6.5045,2132-04-19 16:34:00,2132-05-19 13:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,JEWISH,WIDOWED,WHITE,2132-04-19 14:07:00,2132-04-19 18:02:00,RESPIRATORY DISTRESS,0,1,F,2057-05-11 00:00:00,2139-12-05 00:00:00,2139-12-05 00:00:00,2139-12-05 00:00:00,1,75,2132-04-19 16:34:00,1.0
2,80030,100442,249621,metavision,MICU,MICU,50,50,2119-06-07 02:44:09,2119-06-14 18:38:33,7.6628,2119-06-07 02:43:00,2119-07-18 17:15:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,LONG TERM CARE HOSPITAL,Private,ENGL,CATHOLIC,MARRIED,WHITE,2119-06-06 21:12:00,2119-06-07 03:25:00,HYPOTENSION,0,1,F,2063-07-15 00:00:00,NaN,NaN,NaN,0,56,2119-06-07 02:43:00,1.0
3,85501,100606,234615,metavision,CCU,CCU,7,7,2164-04-01 21:02:44,2164-04-03 21:28:52,2.0181,2164-04-01 21:02:00,2164-04-11 16:15:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,HEART FAILURE,0,1,M,2096-01-10 00:00:00,NaN,NaN,NaN,0,68,2164-04-01 21:02:00,1.0
4,12215,100658,239402,carevue,MICU,MICU,12,12,2186-02-07 05:35:47,2186-02-11 18:22:00,4.5321,2186-02-07 05:34:00,2186-02-14 20:54:00,2186-02-14 20:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,UNOBTAINABLE,WIDOWED,WHITE,2186-02-06 23:23:00,2186-02-07 05:30:00,"HYPERNATREMIA,HYPOTHERMIA",1,1,F,2104-04-29 00:00:00,2186-02-14 00:00:00,2186-02-14 00:00:00,2186-02-14 00:00:00,1,82,2186-02-07 05:34:00,1.0


In [ ]:
### Write results to csv file.
# name of csv file 
filename = "cohort.csv"
    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the header
    csvwriter.writerow(cohort.columns) 
        
    # writing the data rows 
    for i in range(cohort.shape[0]):
      csvwriter.writerow(cohort.iloc[i])

# Get Cohort.

In [ ]:
### Load cohort.
cohort = pd.read_csv('cohort.csv', header=0)
print(cohort.shape, cohort['SUBJECT_ID'].nunique())
n = cohort.shape[0]
#cohort.head()

(5043, 36) 5043


## Age, COPD hx

In [ ]:
### Variable: age bin
age_bin = []

for i in range(cohort.shape[0]):
  if cohort.loc[i]['age'] < 40:
    age_bin.append(0)
  elif cohort.loc[i]['age'] >= 70:
    age_bin.append(2)
  else:
    age_bin.append(1)

cohort['age_bin'] = age_bin


In [ ]:
### COPD
diagnoses_icd = pd.read_csv('DIAGNOSES_ICD.csv', header = 0)

bronchitis_searchgrid = ['4910','4911','49120','49121','49122','4918', '4919']
emphysema_searchgrid = ['4928', '5181','5182','7702']
copd_searchgrid = np.concatenate((bronchitis_searchgrid, emphysema_searchgrid))

copd_dx = diagnoses_icd.loc[np.where(diagnoses_icd['ICD9_CODE'].isin(copd_searchgrid) == True)[0]]
print('No. COPD total: ', len(copd_dx['SUBJECT_ID']))
print('No. COPD unique: ', len(copd_dx['SUBJECT_ID'].unique()))

copd_dx.drop(copd_dx.columns[[0,3,4]], axis = 1, inplace = True)

copd = np.zeros((n,1))
for i in range(n):
  if cohort.loc[i]['HADM_ID'] in copd_dx['HADM_ID']:
    copd[i] = 1
  else: 
    continue

cohort['COPD_hx'] = copd
print(cohort['COPD_hx'].value_counts())
cohort.head()


No. COPD total:  2172
No. COPD unique:  1776
0.0    5018
1.0      25
Name: COPD_hx, dtype: int64


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,age,min,outcome,age_bin,COPD_hx
0,49079,100234,256814,metavision,MICU,MICU,50,50,2118-10-23 19:35:15,2118-10-26 16:08:35,2.8565,2118-10-23 19:34:00,2118-11-05 05:36:00,2118-11-05 05:36:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,2118-10-23 17:04:00,2118-10-23 20:52:00,PNEUMONIA,1,1,M,2035-03-06 00:00:00,2118-11-05 00:00:00,2118-11-05 00:00:00,2118-11-05 00:00:00,1,83,2118-10-23 19:34:00,1.0,2,0.0
1,5476,100343,207651,carevue,MICU,MICU,15,15,2132-04-20 09:20:36,2132-04-26 21:27:06,6.5045,2132-04-19 16:34:00,2132-05-19 13:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,JEWISH,WIDOWED,WHITE,2132-04-19 14:07:00,2132-04-19 18:02:00,RESPIRATORY DISTRESS,0,1,F,2057-05-11 00:00:00,2139-12-05 00:00:00,2139-12-05 00:00:00,2139-12-05 00:00:00,1,75,2132-04-19 16:34:00,1.0,2,0.0
2,80030,100442,249621,metavision,MICU,MICU,50,50,2119-06-07 02:44:09,2119-06-14 18:38:33,7.6628,2119-06-07 02:43:00,2119-07-18 17:15:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,LONG TERM CARE HOSPITAL,Private,ENGL,CATHOLIC,MARRIED,WHITE,2119-06-06 21:12:00,2119-06-07 03:25:00,HYPOTENSION,0,1,F,2063-07-15 00:00:00,NaN,NaN,NaN,0,56,2119-06-07 02:43:00,1.0,1,0.0
3,85501,100606,234615,metavision,CCU,CCU,7,7,2164-04-01 21:02:44,2164-04-03 21:28:52,2.0181,2164-04-01 21:02:00,2164-04-11 16:15:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,HEART FAILURE,0,1,M,2096-01-10 00:00:00,NaN,NaN,NaN,0,68,2164-04-01 21:02:00,1.0,1,0.0
4,12215,100658,239402,carevue,MICU,MICU,12,12,2186-02-07 05:35:47,2186-02-11 18:22:00,4.5321,2186-02-07 05:34:00,2186-02-14 20:54:00,2186-02-14 20:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,UNOBTAINABLE,WIDOWED,WHITE,2186-02-06 23:23:00,2186-02-07 05:30:00,"HYPERNATREMIA,HYPOTHERMIA",1,1,F,2104-04-29 00:00:00,2186-02-14 00:00:00,2186-02-14 00:00:00,2186-02-14 00:00:00,1,82,2186-02-07 05:34:00,1.0,2,0.0


## (TBD) Note variables.

In [ ]:
notes = pd.read_csv('NOTEEVENTS.csv', header = 0)
print(notes.shape)
notes.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2083180, 11)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


## Lab variables prep.

In [ ]:
# Load lab items dictionary.
labitems = pd.read_csv('D_LABITEMS.csv', header = 0)
print(labitems.shape)
labitems.head()

(753, 6)


,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,546,51346,Blasts,Cerebrospinal Fluid (CSF),Hematology,26447-3
1,547,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5
2,548,51348,"Hematocrit, CSF",Cerebrospinal Fluid (CSF),Hematology,30398-2
3,549,51349,Hypersegmented Neutrophils,Cerebrospinal Fluid (CSF),Hematology,26506-6
4,550,51350,Immunophenotyping,Cerebrospinal Fluid (CSF),Hematology,NaN


In [ ]:
# Lab variable IDs.
wbc_id = ['51300','51533','51301']
hct_id = ['51221']
plt_id = ['51265']
cr_id = ['50912']
lac_id = ['50813'] #blood gas
dbili_id = ['50883']
ibili_id = ['50884']
tbili_id = ['50885']
inr_id = ['51237']

labitems[labitems['LABEL'].str.contains('GCS', na = False)]


,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE


In [ ]:
# Load lab events.
labs = pd.read_csv('LABEVENTS.csv', header = 0)
print(labs.shape)
labs.head()

(27854055, 9)


,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal


In [ ]:
# Get variables from labs.
wbc = labs.loc[np.where(labs['ITEMID'].isin(wbc_id) & labs['HADM_ID'].isin(cohort['HADM_ID']) == True)[0]]
wbc = wbc[['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'VALUENUM']]
wbc.columns = ['SUBJECT_ID', 'HADM_ID', 'wbc_time', 'wbc']

hct = labs.loc[np.where(labs['ITEMID'].isin(hct_id) & labs['HADM_ID'].isin(cohort['HADM_ID'])== True)[0]]
hct = hct[['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'VALUENUM']]
hct.columns = ['SUBJECT_ID', 'HADM_ID', 'hct_time', 'hct']

plt = labs.loc[np.where(labs['ITEMID'].isin(plt_id) & labs['HADM_ID'].isin(cohort['HADM_ID'])== True)[0]]
plt = plt[['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'VALUENUM']]
plt.columns = ['SUBJECT_ID', 'HADM_ID', 'plt_time', 'plt']

cr = labs.loc[np.where(labs['ITEMID'].isin(cr_id) & labs['HADM_ID'].isin(cohort['HADM_ID'])== True)[0]]
cr = cr[['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'VALUENUM']]
cr.columns = ['SUBJECT_ID', 'HADM_ID', 'cr_time', 'cr']

lac = labs.loc[np.where(labs['ITEMID'].isin(lac_id) & labs['HADM_ID'].isin(cohort['HADM_ID'])== True)[0]]
lac = lac[['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'VALUENUM']]
lac.columns = ['SUBJECT_ID', 'HADM_ID', 'lac_time', 'lac']

#biliD = labs[labs['ITEMID'] == biliD_id]
#biliI = labs[labs['ITEMID'] == biliI_id]
tbili = labs.loc[np.where(labs['ITEMID'].isin(tbili_id) & labs['HADM_ID'].isin(cohort['HADM_ID'])== True)[0]]
tbili = tbili[['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'VALUENUM']]
tbili.columns = ['SUBJECT_ID', 'HADM_ID', 'tbili_time', 'tbili']

inr = labs.loc[np.where(labs['ITEMID'].isin(inr_id) & labs['HADM_ID'].isin(cohort['HADM_ID'])== True)[0]]
inr = inr[['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'VALUENUM']]
inr.columns = ['SUBJECT_ID', 'HADM_ID', 'inr_time', 'inr']

print(wbc.shape, hct.shape, plt.shape, cr.shape, lac.shape, tbili.shape, inr.shape)

(95214, 4) (110847, 4) (98654, 4) (109363, 4) (42592, 4) (31460, 4) (64873, 4)


In [ ]:
### Write results to csv file.
# name of csv file 
#filename = "wbc.csv"
    
# writing to csv file 
#with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    #csvwriter = csv.writer(csvfile) 
        
    # writing the header
    #csvwriter.writerow(wbc.columns) 
        
    # writing the data rows 
    #for i in range(wbc.shape[0]):
      #csvwriter.writerow(wbc.iloc[i])

## Lab variables processing.

In [ ]:
#wbc = pd.read_csv('wbc.csv', header = 0)
#hct = pd.read_csv('hct.csv', header = 0)
#plt = pd.read_csv('plt.csv', header = 0)
#cr = pd.read_csv('cr.csv', header = 0)
#lac = pd.read_csv('lac.csv', header = 0)
#bili = pd.read_csv('bili.csv', header = 0)
#inr = pd.read_csv('inr.csv', header = 0)

In [ ]:
### Last WBC.
cohort_labs = pd.merge(cohort, wbc, how = 'left', on = ['SUBJECT_ID', 'HADM_ID'])
cohort_labs = cohort_labs.sort_values(['HADM_ID', 'wbc_time'])
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

cohort_labs['wbc_time'].fillna('0', inplace = True)

drop = []
change_val = []
for i in range(cohort_labs.shape[0]):
  time_cutoff = cohort_labs.loc[i]['OUTTIME'] 
  lab_time = cohort_labs.loc[i]['wbc_time']
  if time_cutoff < lab_time:
    if cohort_labs.loc[i]['HADM_ID'] != cohort_labs.loc[i-1]['HADM_ID'] & cohort_labs.loc[i]['HADM_ID'] != cohort_labs.loc[i+1]['HADM_ID']:
      change_val.append(i)
    else: 
      drop.append(i)
  else:
    continue 

for i in range(len(change_val)):
  cohort_labs.loc[change_val[i],['wbc']] = 'NaN'

cohort_labs = cohort_labs.drop(drop)
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

print(cohort_labs.shape[0])

cohort_labs = cohort_labs.sort_values(['HADM_ID','wbc_time'])
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

keep = []
for i in range(cohort_labs.shape[0]-1):
  if cohort_labs.loc[i]['HADM_ID'] == cohort_labs.loc[i+1]['HADM_ID']:
    continue
  else: 
    keep.append(i)

keep.append(cohort_labs.shape[0]-1)

cohort_labs = cohort_labs.iloc[keep]
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

print(cohort_labs.shape[0])
#cohort_labs.head()

71804
5043


In [ ]:
### Last Hct.
cohort_labs = pd.merge(cohort_labs, hct, how = 'left', on = ['SUBJECT_ID', 'HADM_ID'])
cohort_labs = cohort_labs.sort_values(['HADM_ID', 'hct_time'])
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

cohort_labs['hct_time'].fillna('0', inplace = True)

drop = []
change_val = []
for i in range(cohort_labs.shape[0]):
  time_cutoff = cohort_labs.loc[i]['OUTTIME'] 
  lab_time = cohort_labs.loc[i]['hct_time']
  if time_cutoff < lab_time:
    if cohort_labs.loc[i]['HADM_ID'] != cohort_labs.loc[i-1]['HADM_ID'] & cohort_labs.loc[i]['HADM_ID'] != cohort_labs.loc[i+1]['HADM_ID']:
      change_val.append(i)
    else: 
      drop.append(i)
  else:
    continue 

for i in range(len(change_val)):
  cohort_labs.loc[change_val[i],['hct']] = 'NaN'

cohort_labs = cohort_labs.drop(drop)
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

print(cohort_labs.shape[0])

cohort_labs = cohort_labs.sort_values(['HADM_ID','hct_time'])
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

keep = []
for i in range(cohort_labs.shape[0]-1):
  if cohort_labs.loc[i]['HADM_ID'] == cohort_labs.loc[i+1]['HADM_ID']:
    continue
  else: 
    keep.append(i)

keep.append(cohort_labs.shape[0]-1)

cohort_labs = cohort_labs.iloc[keep]
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

print(cohort_labs.shape[0])
#cohort_labs.head()

84983
5043


In [ ]:
### Last plt.
cohort_labs = pd.merge(cohort_labs, plt, how = 'left', on = ['SUBJECT_ID', 'HADM_ID'])
cohort_labs = cohort_labs.sort_values(['HADM_ID', 'plt_time'])
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

cohort_labs['plt_time'].fillna('0', inplace = True)

drop = []
change_val = []
for i in range(cohort_labs.shape[0]):
  time_cutoff = cohort_labs.loc[i]['OUTTIME'] 
  lab_time = cohort_labs.loc[i]['plt_time']
  if time_cutoff < lab_time:
    if cohort_labs.loc[i]['HADM_ID'] != cohort_labs.loc[i-1]['HADM_ID'] & cohort_labs.loc[i]['HADM_ID'] != cohort_labs.loc[i+1]['HADM_ID']:
      change_val.append(i)
    else: 
      drop.append(i)
  else:
    continue 

for i in range(len(change_val)):
  cohort_labs.loc[change_val[i],['plt']] = 'NaN'

cohort_labs = cohort_labs.drop(drop)
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

print(cohort_labs.shape[0])

cohort_labs = cohort_labs.sort_values(['HADM_ID','plt_time'])
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

keep = []
for i in range(cohort_labs.shape[0]-1):
  if cohort_labs.loc[i]['HADM_ID'] == cohort_labs.loc[i+1]['HADM_ID']:
    continue
  else: 
    keep.append(i)

keep.append(cohort_labs.shape[0]-1)

cohort_labs = cohort_labs.iloc[keep]
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

print(cohort_labs.shape[0])
#cohort_labs.head()

74630
5043


In [ ]:
### Last Cr.
cohort_labs = pd.merge(cohort_labs, cr, how = 'left', on = ['SUBJECT_ID', 'HADM_ID'])
cohort_labs = cohort_labs.sort_values(['HADM_ID', 'cr_time'])
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

cohort_labs['cr_time'].fillna('0', inplace = True)

drop = []
change_val = []
for i in range(cohort_labs.shape[0]):
  time_cutoff = cohort_labs.loc[i]['OUTTIME'] 
  lab_time = cohort_labs.loc[i]['cr_time']
  if time_cutoff < lab_time:
    if cohort_labs.loc[i]['HADM_ID'] != cohort_labs.loc[i-1]['HADM_ID'] & cohort_labs.loc[i]['HADM_ID'] != cohort_labs.loc[i+1]['HADM_ID']:
      change_val.append(i)
    else: 
      drop.append(i)
  else:
    continue 

for i in range(len(change_val)):
  cohort_labs.loc[change_val[i],['cr']] = 'NaN'

cohort_labs = cohort_labs.drop(drop)
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

print(cohort_labs.shape[0])

cohort_labs = cohort_labs.sort_values(['HADM_ID','cr_time'])
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

keep = []
for i in range(cohort_labs.shape[0]-1):
  if cohort_labs.loc[i]['HADM_ID'] == cohort_labs.loc[i+1]['HADM_ID']:
    continue
  else: 
    keep.append(i)

keep.append(cohort_labs.shape[0]-1)

cohort_labs = cohort_labs.iloc[keep]
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

print(cohort_labs.shape[0])
#cohort_labs.head()

84049
5043


In [ ]:
### Last lactate.
cohort_labs = pd.merge(cohort_labs, lac, how = 'left', on = ['SUBJECT_ID', 'HADM_ID'])
cohort_labs = cohort_labs.sort_values(['HADM_ID', 'lac_time'])
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

cohort_labs['lac_time'].fillna('0', inplace = True)

drop = []
change_val = []
for i in range(cohort_labs.shape[0]):
  time_cutoff = cohort_labs.loc[i]['OUTTIME'] 
  lab_time = cohort_labs.loc[i]['lac_time']
  if time_cutoff < lab_time:
    if cohort_labs.loc[i]['HADM_ID'] != cohort_labs.loc[i-1]['HADM_ID'] & cohort_labs.loc[i]['HADM_ID'] != cohort_labs.loc[i+1]['HADM_ID']:
      change_val.append(i)
    else: 
      drop.append(i)
  else:
    continue 

for i in range(len(change_val)):
  cohort_labs.loc[change_val[i],['lac']] = 'NaN'

cohort_labs = cohort_labs.drop(drop)
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

print(cohort_labs.shape[0])

cohort_labs = cohort_labs.sort_values(['HADM_ID','lac_time'])
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

keep = []
for i in range(cohort_labs.shape[0]-1):
  if cohort_labs.loc[i]['HADM_ID'] == cohort_labs.loc[i+1]['HADM_ID']:
    continue
  else: 
    keep.append(i)

keep.append(cohort_labs.shape[0]-1)

cohort_labs = cohort_labs.iloc[keep]
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

print(cohort_labs.shape[0])


39846
5043


In [ ]:
### Last total bilirubin.
cohort_labs = pd.merge(cohort_labs, tbili, how = 'left', on = ['SUBJECT_ID', 'HADM_ID'])
cohort_labs = cohort_labs.sort_values(['HADM_ID', 'tbili_time'])
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

cohort_labs['tbili_time'].fillna('0', inplace = True)

drop = []
change_val = []
for i in range(cohort_labs.shape[0]):
  time_cutoff = cohort_labs.loc[i]['OUTTIME'] 
  lab_time = cohort_labs.loc[i]['tbili_time']
  if time_cutoff < lab_time:
    if cohort_labs.loc[i]['HADM_ID'] != cohort_labs.loc[i-1]['HADM_ID'] & cohort_labs.loc[i]['HADM_ID'] != cohort_labs.loc[i+1]['HADM_ID']:
      change_val.append(i)
    else: 
      drop.append(i)
  else:
    continue 

for i in range(len(change_val)):
  cohort_labs.loc[change_val[i],['tbili']] = 'NaN'

cohort_labs = cohort_labs.drop(drop)
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

print(cohort_labs.shape[0])

cohort_labs = cohort_labs.sort_values(['HADM_ID','tbili_time'])
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

keep = []
for i in range(cohort_labs.shape[0]-1):
  if cohort_labs.loc[i]['HADM_ID'] == cohort_labs.loc[i+1]['HADM_ID']:
    continue
  else: 
    keep.append(i)

keep.append(cohort_labs.shape[0]-1)

cohort_labs = cohort_labs.iloc[keep]
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

print(cohort_labs.shape[0])
#cohort_labs.head()

25182
5043


In [ ]:
### Last INR.
cohort_labs = pd.merge(cohort_labs, inr, how = 'left', on = ['SUBJECT_ID', 'HADM_ID'])
cohort_labs = cohort_labs.sort_values(['HADM_ID', 'inr_time'])
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

cohort_labs['inr_time'].fillna('0', inplace = True)

drop = []
change_val = []
for i in range(cohort_labs.shape[0]):
  time_cutoff = cohort_labs.loc[i]['OUTTIME'] 
  lab_time = cohort_labs.loc[i]['inr_time']
  if time_cutoff < lab_time:
    if cohort_labs.loc[i]['HADM_ID'] != cohort_labs.loc[i-1]['HADM_ID'] & cohort_labs.loc[i]['HADM_ID'] != cohort_labs.loc[i+1]['HADM_ID']:
      change_val.append(i)
    else: 
      drop.append(i)
  else:
    continue 

for i in range(len(change_val)):
  cohort_labs.loc[change_val[i],['inr']] = 'NaN'

cohort_labs = cohort_labs.drop(drop)
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

print(cohort_labs.shape[0])

cohort_labs = cohort_labs.sort_values(['HADM_ID','inr_time'])
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

keep = []
for i in range(cohort_labs.shape[0]-1):
  if cohort_labs.loc[i]['HADM_ID'] == cohort_labs.loc[i+1]['HADM_ID']:
    continue
  else: 
    keep.append(i)

keep.append(cohort_labs.shape[0]-1)

cohort_labs = cohort_labs.iloc[keep]
cohort_labs = cohort_labs.reset_index()
del cohort_labs['index']

print(cohort_labs.shape[0])
#cohort_labs.head()

51662
5043


## (RUN ONCE) MAP, PaO2, FiO2, SpO2, HR, temp, GCS, RASS, vent type csv prep.


In [ ]:
### Examine other csv files for data curation.
items = pd.read_csv('D_ITEMS.csv', header = 0)
print(items.shape)
items.head()

(12487, 10)


,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,457,497,Patient controlled analgesia (PCA) [Inject],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,458,498,PCA Lockout (Min),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,459,499,PCA Medication,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,460,500,PCA Total Dose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,461,501,PCV Exh Vt (Obser),NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [ ]:
### Search for variable ITEMID codes.
paco2_id = ['778'] #ABG
sao2_id = ['834'] #ABG
ph_abg_id = ['780'] #ABG
hco3_abg_id = ['812'] #ABG

pvco2_id = ['3830'] #VBG
pvo2_id = ['3831'] #VBG

#fio2_id = [2981,3420,7018,7041,227009,227010,226754,223835]

pao2_id = [779,220224] #ABG
fio2_id = [3420,223835]
hr_id = [211,220045]
spo2_id = [646,220277]
vent_type_id = [722,223849]
gcs = [198,220739,223900,223901]


temp_id = [678,223761]
rass_id = [228302, 228096]
map_id = [438, 672, 3067, 5680, 5804, 2309, 2353, 2544, 2770, 2974,6399, 6579, 6605, 227023]


items[items['LABEL'].str.contains('MAP', na = False)]
#items[items['ITEMID'] == 220227]
#items.head()

#item_vent_searchgrid = ['1736', '1738', '1976','1990','5874','1487','1577','1659','1697','2034','2124','6115','6130','6185','225411'] #Item IDs for invasive mechanical ventilation.


In [ ]:
### Cycle through chartevents.
chartevents = pd.read_csv('/content/drive/MyDrive/BMI212/chartevents/CHARTEVENTS_15.csv', header = 0)
print(chartevents.shape)
#chartevents.head()

(19999999, 15)


In [ ]:
temp_cohort = chartevents.loc[np.where(chartevents['HADM_ID'].isin(cohort['HADM_ID']) == True)[0]]
print(temp_cohort.shape[0], temp_cohort['SUBJECT_ID'].nunique())
#temp_cohort.head()
#wbc = wbc[['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'VALUENUM']]
#wbc.columns = ['SUBJECT_ID', 'HADM_ID', 'wbc_time', 'wbc']

5829878 675


In [ ]:
# Get variables: HR, SpO2, PaO2, FiO2
#searchgrid = np.concatenate((hr_id, spo2_id, pao2_id, fio2_id))
#searchgrid = map(int, searchgrid)
pao2_id = [779,220224] #ABG
fio2_id = [3420,223835]
spo2_id = [646,220277]
hr_id = [211,220045]

gcs_id = [198,220739,223900,223901]
vent_type_id = [722,223849]

temp_id = [678,223761]
rass_id = [228302,228096]
map_id = [438,672,3067,5680,5804,2309,2353,2544,2770,2974,6399, 6579,6605,227023]

temp_chartevents = temp_cohort.iloc[np.where(temp_cohort['ITEMID'].isin(temp_id) == True)[0]]
#items[items['LABEL'].str.contains('lac', na = False) & items['DBSOURCE'].str.contains('metavision', na=False)]

print(temp_chartevents.shape[0], temp_chartevents['SUBJECT_ID'].nunique())
#temp_chartevents['VALUE'].value_counts()
#temp_chartevents.head()

# REMOVE (containing CPAP): CPAP/PSV, Standby, CPAP/PSV+ApnVol, CPAP/PPS, CPAP, 
# count intubation days via unique charttime days

0 0


In [ ]:
### Write results to csv file.
# name of csv file 
filename = "rass15.csv"
    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the header
    csvwriter.writerow(temp_chartevents.columns) 
        
    # writing the data rows 
    for i in range(temp_chartevents.shape[0]):
      csvwriter.writerow(temp_chartevents.iloc[i])

## ABG, VS, clinical scores.

In [ ]:
### FiO2.
fio20 = pd.read_csv('fio20.csv', header = 0)
fio21 = pd.read_csv('fio21.csv', header = 0)
fio26 = pd.read_csv('fio26.csv', header = 0)

fio2 = pd.concat([fio20,fio21,fio26])
fio2 = fio2[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'VALUENUM']]
fio2.columns = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID','fio2_time', 'fio2']
print(fio2.shape)
fio2.head()

(197153, 5)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,fio2_time,fio2
0,188,123860,213646.0,2160-12-31 14:00:00,40.0
1,188,123860,213646.0,2160-12-31 16:00:00,40.0
2,188,123860,213646.0,2161-01-02 08:00:00,40.0
3,188,123860,213646.0,2161-01-02 10:30:00,35.0
4,188,123860,213646.0,2160-12-31 20:00:00,40.0


In [ ]:
### Get last FiO2. 
features = pd.merge(cohort_labs, fio2, how = 'left', on = ['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
features = features.sort_values(['HADM_ID', 'fio2_time'])
features = features.reset_index()
del features['index']

features['fio2_time'].fillna('0', inplace = True)

drop = []
change_val = []
for i in range(features.shape[0]):
  time_cutoff = features.loc[i]['OUTTIME'] 
  var_time = features.loc[i]['fio2_time']
  if time_cutoff < var_time:
    if features.loc[i]['HADM_ID'] != features.loc[i-1]['HADM_ID'] & features.loc[i]['HADM_ID'] != features.loc[i+1]['HADM_ID']:
      change_val.append(i)
    else: 
      drop.append(i)
  else:
    continue 

for i in range(len(change_val)):
  features.loc[change_val[i],['fio2']] = 'NaN'

features = features.drop(drop)
features = features.reset_index()
del features['index']

print(features.shape[0])

features = features.sort_values(['HADM_ID','fio2_time'])
features = features.reset_index()
del features['index']

keep = []
for i in range(features.shape[0]-1):
  if features.loc[i]['HADM_ID'] == features.loc[i+1]['HADM_ID']:
    continue
  else: 
    keep.append(i)

keep.append(features.shape[0]-1)

features = features.iloc[keep]
features = features.reset_index()
del features['index']

print(features.shape[0])

160367
5043


In [ ]:
### PaO2
pao20 = pd.read_csv('pao20.csv', header = 0)
pao21 = pd.read_csv('pao21.csv', header = 0)
pao22 = pd.read_csv('pao22.csv', header = 0)
pao23 = pd.read_csv('pao23.csv', header = 0)
pao24 = pd.read_csv('pao24.csv', header = 0)
pao25 = pd.read_csv('pao25.csv', header = 0)
pao26 = pd.read_csv('pao26.csv', header = 0)
pao27 = pd.read_csv('pao27.csv', header = 0)
pao28 = pd.read_csv('pao28.csv', header = 0)
pao29 = pd.read_csv('pao29.csv', header = 0)

pao2 = pd.concat([pao20,pao21,pao22,pao23,pao24,pao25,pao26,pao27,pao28,pao29])
pao2 = pao2[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'VALUENUM']]
pao2.columns = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID','pao2_time', 'pao2']
print(pao2.shape)
pao2.head()

(112749, 5)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,pao2_time,pao2
0,188,123860,213646.0,2160-12-31 13:15:00,173.0
1,188,123860,213646.0,2161-01-01 04:00:00,144.0
2,502,116367,209737.0,2143-10-24 11:20:00,100.0
3,502,116367,209737.0,2143-10-24 04:28:00,110.0
4,502,116367,209737.0,2143-10-24 05:49:00,95.0


In [ ]:
### Get last PaO2. 
features = pd.merge(features, pao2, how = 'left', on = ['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
features = features.sort_values(['HADM_ID', 'pao2_time'])
features = features.reset_index()
del features['index']

features['pao2_time'].fillna('0', inplace = True)

drop = []
change_val = []
for i in range(features.shape[0]):
  time_cutoff = features.loc[i]['OUTTIME'] 
  var_time = features.loc[i]['pao2_time']
  if time_cutoff < var_time:
    if features.loc[i]['HADM_ID'] != features.loc[i-1]['HADM_ID'] & features.loc[i]['HADM_ID'] != features.loc[i+1]['HADM_ID']:
      change_val.append(i)
    else: 
      drop.append(i)
  else:
    continue 

for i in range(len(change_val)):
  features.loc[change_val[i],['pao2']] = 'NaN'

features = features.drop(drop)
features = features.reset_index()
del features['index']

print(features.shape[0])

features = features.sort_values(['HADM_ID','pao2_time'])
features = features.reset_index()
del features['index']

keep = []
for i in range(features.shape[0]-1):
  if features.loc[i]['HADM_ID'] == features.loc[i+1]['HADM_ID']:
    continue
  else: 
    keep.append(i)

keep.append(features.shape[0]-1)

features = features.iloc[keep]
features = features.reset_index()
del features['index']

print(features.shape[0])

104302
5043


In [ ]:
### Get last PaO2/FiO2.
features['pafio2'] = features['pao2']/features['fio2']
features.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,age,min,outcome,age_bin,COPD_hx,wbc_time,wbc,hct_time,hct,plt_time,plt,cr_time,cr,lac_time,lac,tbili_time,tbili,inr_time,inr,fio2_time,fio2,pao2_time,pao2,pafio2
0,68591,100016,217590,metavision,MICU,MICU,52,23,2188-05-24 13:07:20,2188-05-30 17:16:33,6.1731,2188-05-24 13:06:00,2188-05-30 17:16:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,PROTESTANT QUAKER,SINGLE,WHITE,2188-05-24 11:11:00,2188-05-24 15:00:00,PNEUMONIA,0,1,M,2132-11-19 00:00:00,2188-07-06 00:00:00,NaN,2188-07-06 00:00:00,1,56,2188-05-24 13:06:00,0.0,1,0.0,2188-05-30 02:56:00,8.1,2188-05-30 02:56:00,26.7,2188-05-30 02:56:00,130,2188-05-30 02:56:00,1.1,2188-05-24 11:00:00,2,0,NaN,2188-05-27 02:50:00,1.7,2188-05-30 16:00:00,40.0,2188-05-24 18:24:00,86.0,2.150000
1,48539,100035,245719,metavision,MICU,MICU,50,50,2115-02-22 06:52:06,2115-03-04 19:00:50,10.5061,2115-02-22 06:51:00,2115-03-19 14:37:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,REHAB/DISTINCT PART HOSP,Medicaid,SPAN,NOT SPECIFIED,SINGLE,HISPANIC OR LATINO,2115-02-22 06:24:00,2115-02-22 09:14:00,POST ARREST,0,1,M,2078-08-09 00:00:00,NaN,NaN,NaN,0,37,2115-02-22 06:51:00,0.0,0,0.0,2115-03-04 04:56:00,21.2,2115-03-04 04:56:00,41.7,2115-03-04 04:56:00,203,2115-03-04 04:56:00,1.2,2115-02-25 07:59:00,1,2115-02-24 03:34:00,0.3,2115-02-27 03:02:00,1.2,2115-03-02 03:00:00,70.0,2115-02-28 04:56:00,95.0,1.357143
2,69282,100063,241782,metavision,TSICU,MICU,14,23,2181-04-21 00:53:05,2181-04-22 16:29:17,1.6501,2181-04-20 12:00:00,2181-04-22 23:59:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,SPAN,CATHOLIC,SINGLE,HISPANIC OR LATINO,2181-04-20 18:11:00,2181-04-21 03:54:00,SUICIDAL IDEATION;PSYCH EVAL,0,1,M,2135-04-05 00:00:00,NaN,NaN,NaN,0,46,2181-04-20 12:00:00,0.0,1,0.0,2181-04-22 03:09:00,9.3,2181-04-22 03:09:00,40.9,2181-04-22 03:09:00,183,2181-04-22 03:09:00,1,0,NaN,2181-04-22 03:09:00,0.4,2181-04-22 03:09:00,1,2181-04-21 13:00:00,50.0,2181-04-21 01:35:00,365.0,7.300000
3,42255,100087,294404,metavision,CCU,CCU,7,7,2126-11-01 17:47:40,2126-11-05 11:17:03,3.7287,2126-11-01 17:47:00,2126-11-06 05:00:00,2126-11-06 05:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,CARDIAC ARREST,1,1,M,2069-11-06 00:00:00,2126-11-06 00:00:00,2126-11-06 00:00:00,2126-11-06 00:00:00,1,57,2126-11-01 17:47:00,0.0,1,0.0,2126-11-04 04:19:00,8.5,2126-11-04 04:19:00,44.8,2126-11-04 04:19:00,215,2126-11-04 04:19:00,0.7,2126-11-02 01:02:00,1.3,2126-11-03 04:26:00,2.4,2126-11-04 04:19:00,1.5,2126-11-04 12:00:00,30.0,2126-11-03 05:12:00,143.0,4.766667
4,19984,100132,229201,carevue,MICU,MICU,52,52,2179-03-05 20:14:43,2179-03-24 11:37:33,18.6409,2179-03-05 16:36:00,2179-03-24 11:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,Medicare,NaN,JEWISH,MARRIED,BLACK/AFRICAN AMERICAN,2179-03-05 14:26:00,2179-03-05 22:36:00,DYSPNEA,0,1,F,2105-03-13 00:00:00,2179-05-07 00:00:00,NaN,2179-05-07 00:00:00,1,74,2179-03-05 16:36:00,0.0,2,0.0,2179-03-23 01:44:00,10,2179-03-23 01:44:00,27.6,2179-03-23 01:44:00,316,2179-03-23 01:44:00,1.3,2179-03-22 15:11:00,1.2,2179-03-19 04:54:00,0.3,2179-03-22 03:15:00,1,0,NaN,2179-03-07 15:41:00,97.0,NaN


In [ ]:
hr1 = pd.read_csv('hr1.csv', header=0)
hr2 = pd.read_csv('hr2.csv', header=0)
hr3 = pd.read_csv('hr3.csv', header=0)
hr4 = pd.read_csv('hr4.csv', header=0)
hr5 = pd.read_csv('hr5.csv', header=0)
hr6 = pd.read_csv('hr6.csv', header=0)
hr7 = pd.read_csv('hr7.csv', header=0)
hr8 = pd.read_csv('hr8.csv', header=0)
hr9 = pd.read_csv('hr9.csv', header=0)

hr = pd.concat([hr1, hr2, hr3, hr4, hr5, hr6, hr7, hr8, hr9])
hr = hr[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'VALUENUM']]
hr.columns = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID','hr_time', 'hr']
print(hr.shape)
hr.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1099361, 5)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,hr_time,hr
0,68564,101945,209221.0,2119-08-18 14:00:00,100.0
1,68564,101945,209221.0,2119-08-18 15:00:00,95.0
2,68564,101945,209221.0,2119-08-18 16:00:00,92.0
3,68564,101945,209221.0,2119-08-18 17:00:00,89.0
4,68564,101945,209221.0,2119-08-18 18:00:00,92.0


In [ ]:
### Get last HR. 
features = pd.merge(features, hr, how = 'left', on = ['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
features = features.sort_values(['HADM_ID', 'hr_time'])
features = features.reset_index()
del features['index']

features['hr_time'].fillna('0', inplace = True)

drop = []
change_val = []
for i in range(features.shape[0]):
  time_cutoff = features.loc[i]['OUTTIME'] 
  var_time = features.loc[i]['hr_time']
  if time_cutoff < var_time:
    if features.loc[i]['HADM_ID'] != features.loc[i-1]['HADM_ID'] & features.loc[i]['HADM_ID'] != features.loc[i+1]['HADM_ID']:
      change_val.append(i)
    else: 
      drop.append(i)
  else:
    continue 

for i in range(len(change_val)):
  features.loc[change_val[i],['hr']] = 'NaN'

features = features.drop(drop)
features = features.reset_index()
del features['index']

print(features.shape[0])

features = features.sort_values(['HADM_ID','hr_time'])
features = features.reset_index()
del features['index']

keep = []
for i in range(features.shape[0]-1):
  if features.loc[i]['HADM_ID'] == features.loc[i+1]['HADM_ID']:
    continue
  else: 
    keep.append(i)

keep.append(features.shape[0]-1)

features = features.iloc[keep]
features = features.reset_index()
del features['index']

print(features.shape[0])

981903
5043


In [ ]:
### Temp
temp0 = pd.read_csv('temp0.csv', header=0)
temp1 = pd.read_csv('temp1.csv', header=0)
temp2 = pd.read_csv('temp2.csv', header=0)
temp3 = pd.read_csv('temp3.csv', header=0)
temp4 = pd.read_csv('temp4.csv', header=0)
temp5 = pd.read_csv('temp5.csv', header=0)
temp6 = pd.read_csv('temp6.csv', header=0)
temp7 = pd.read_csv('temp7.csv', header=0)
temp8 = pd.read_csv('temp8.csv', header=0)
temp9 = pd.read_csv('temp9.csv', header=0)

temp = pd.concat([temp0, temp1, temp2, temp3, temp4, temp5, temp6, temp7, temp8, temp9])
temp = temp[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'VALUENUM']]
temp.columns = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID','temp_time', 'temp']
print(temp.shape)
temp.head()


(324006, 5)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,temp_time,temp
0,188,123860,213646.0,2161-01-02 08:00:00,96.0
1,188,123860,213646.0,2160-12-31 18:00:00,97.5
2,188,123860,213646.0,2160-12-31 20:00:00,98.6
3,188,123860,213646.0,2161-01-02 12:00:00,96.7
4,188,123860,213646.0,2161-01-02 16:00:00,96.7


In [ ]:
### Get last temp. 
features = pd.merge(features, temp, how = 'left', on = ['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
features = features.sort_values(['HADM_ID', 'temp_time'])
features = features.reset_index()
del features['index']

features['temp_time'].fillna('0', inplace = True)

drop = []
change_val = []
for i in range(features.shape[0]):
  time_cutoff = features.loc[i]['OUTTIME'] 
  var_time = features.loc[i]['temp_time']
  if time_cutoff < var_time:
    if features.loc[i]['HADM_ID'] != features.loc[i-1]['HADM_ID'] & features.loc[i]['HADM_ID'] != features.loc[i+1]['HADM_ID']:
      change_val.append(i)
    else: 
      drop.append(i)
  else:
    continue 

for i in range(len(change_val)):
  features.loc[change_val[i],['temp']] = 'NaN'

features = features.drop(drop)
features = features.reset_index()
del features['index']

print(features.shape[0])

features = features.sort_values(['HADM_ID','temp_time'])
features = features.reset_index()
del features['index']

keep = []
for i in range(features.shape[0]-1):
  if features.loc[i]['HADM_ID'] == features.loc[i+1]['HADM_ID']:
    continue
  else: 
    keep.append(i)

keep.append(features.shape[0]-1)

features = features.iloc[keep]
features = features.reset_index()
del features['index']

print(features.shape[0])

297258
5043


In [ ]:
### SpO2
spo20 = pd.read_csv('spo20.csv', header = 0)
spo21 = pd.read_csv('spo21.csv', header = 0)
spo22 = pd.read_csv('spo22.csv', header = 0)
spo23 = pd.read_csv('spo23.csv', header = 0)
spo24 = pd.read_csv('spo24.csv', header = 0)
spo25 = pd.read_csv('spo25.csv', header = 0)
spo26 = pd.read_csv('spo26.csv', header = 0)
spo27 = pd.read_csv('spo27.csv', header = 0)
spo28 = pd.read_csv('spo28.csv', header = 0)
spo29 = pd.read_csv('spo29.csv', header = 0)

spo2 = pd.concat([spo20,spo21,spo22,spo23,spo24,spo25,spo26,spo27,spo28,spo29])
spo2 = spo2[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'VALUENUM']]
spo2.columns = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID','spo2_time', 'spo2']
print(spo2.shape)
spo2.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1450859, 5)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,spo2_time,spo2
0,188,123860,213646.0,2161-01-02 07:00:00,100.0
1,188,123860,213646.0,2160-12-31 13:44:00,100.0
2,188,123860,213646.0,2160-12-31 14:00:00,100.0
3,188,123860,213646.0,2160-12-31 15:00:00,100.0
4,188,123860,213646.0,2160-12-31 16:00:00,100.0


In [ ]:
### Get last SpO2. 
features = pd.merge(features, spo2, how = 'left', on = ['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
features = features.sort_values(['HADM_ID', 'spo2_time'])
features = features.reset_index()
del features['index']

features['spo2_time'].fillna('0', inplace = True)

drop = []
change_val = []
for i in range(features.shape[0]):
  time_cutoff = features.loc[i]['OUTTIME'] 
  var_time = features.loc[i]['spo2_time']
  if time_cutoff < var_time:
    if features.loc[i]['HADM_ID'] != features.loc[i-1]['HADM_ID'] & features.loc[i]['HADM_ID'] != features.loc[i+1]['HADM_ID']:
      change_val.append(i)
    else: 
      drop.append(i)
  else:
    continue 

for i in range(len(change_val)):
  features.loc[change_val[i],['spo2']] = 'NaN'

features = features.drop(drop)
features = features.reset_index()
del features['index']

print(features.shape[0])

features = features.sort_values(['HADM_ID','spo2_time'])
features = features.reset_index()
del features['index']

keep = []
for i in range(features.shape[0]-1):
  if features.loc[i]['HADM_ID'] == features.loc[i+1]['HADM_ID']:
    continue
  else: 
    keep.append(i)

keep.append(features.shape[0]-1)

features = features.iloc[keep]
features = features.reset_index()
del features['index']

print(features.shape[0])

1306378
5043


In [ ]:
### MAP.
map1 = pd.read_csv('map1.csv', header=0)
map2 = pd.read_csv('map2.csv', header=0)
map3 = pd.read_csv('map3.csv', header=0)
map4 = pd.read_csv('map4.csv', header=0)
map5 = pd.read_csv('map5.csv', header=0)
map6 = pd.read_csv('map6.csv', header=0)
map7 = pd.read_csv('map7.csv', header=0)
map8 = pd.read_csv('map8.csv', header=0)
map9 = pd.read_csv('map9.csv', header=0)

map = pd.concat([map1, map2, map3, map4, map5, map6, map7, map8, map9])
map = map[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'VALUENUM']]
map.columns = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID','map_time', 'map']
print(map.shape)
map.head()


(3351, 5)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,map_time,map
0,78076,138363,257336.0,2113-07-04 15:57:00,122.0
1,82326,127147,263318.0,2105-12-27 19:31:00,55.0
2,286,106909,260225.0,2176-02-23 04:00:00,13.0
3,286,106909,260225.0,2176-01-12 04:00:00,12.0
4,286,106909,260225.0,2176-01-17 07:35:00,12.0


In [ ]:
### Get last MAP. 
features = pd.merge(features, map, how = 'left', on = ['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
features = features.sort_values(['HADM_ID', 'map_time'])
features = features.reset_index()
del features['index']

features['map_time'].fillna('0', inplace = True)

drop = []
change_val = []
for i in range(features.shape[0]):
  time_cutoff = features.loc[i]['OUTTIME'] 
  var_time = features.loc[i]['map_time']
  if time_cutoff < var_time:
    if features.loc[i]['HADM_ID'] != features.loc[i-1]['HADM_ID'] & features.loc[i]['HADM_ID'] != features.loc[i+1]['HADM_ID']:
      change_val.append(i)
    else: 
      drop.append(i)
  else:
    continue 

for i in range(len(change_val)):
  features.loc[change_val[i],['map']] = 'NaN'

features = features.drop(drop)
features = features.reset_index()
del features['index']

print(features.shape[0])

features = features.sort_values(['HADM_ID','map_time'])
features = features.reset_index()
del features['index']

keep = []
for i in range(features.shape[0]-1):
  if features.loc[i]['HADM_ID'] == features.loc[i+1]['HADM_ID']:
    continue
  else: 
    keep.append(i)

keep.append(features.shape[0]-1)

features = features.iloc[keep]
features = features.reset_index()
del features['index']

print(features.shape[0])

8021
5043


In [ ]:
### RASS.
rass13 = pd.read_csv('rass13.csv', header=0)
rass14 = pd.read_csv('rass14.csv', header=0)
rass15 = pd.read_csv('rass15.csv', header=0)

rass = pd.concat([rass13, rass14, rass15])
rass = rass[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'VALUENUM']]
rass.columns = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID','rass_time', 'rass']
print(rass.shape)
rass.head()

(53051, 5)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,rass_time,rass
0,878,102365,239862.0,2137-10-30 20:00:00,0.0
1,878,102365,239862.0,2137-10-31 00:00:00,0.0
2,878,102365,239862.0,2137-10-29 19:38:00,0.0
3,878,102365,239862.0,2137-10-28 10:00:00,-1.0
4,878,102365,239862.0,2137-10-28 12:00:00,1.0


In [ ]:
### Get last RASS. 
features = pd.merge(features, rass, how = 'left', on = ['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
features = features.sort_values(['HADM_ID', 'rass_time'])
features = features.reset_index()
del features['index']

features['rass_time'].fillna('0', inplace = True)

drop = []
change_val = []
for i in range(features.shape[0]):
  time_cutoff = features.loc[i]['OUTTIME'] 
  var_time = features.loc[i]['rass_time']
  if time_cutoff < var_time:
    if features.loc[i]['HADM_ID'] != features.loc[i-1]['HADM_ID'] & features.loc[i]['HADM_ID'] != features.loc[i+1]['HADM_ID']:
      change_val.append(i)
    else: 
      drop.append(i)
  else:
    continue 

for i in range(len(change_val)):
  features.loc[change_val[i],['rass']] = 'NaN'

features = features.drop(drop)
features = features.reset_index()
del features['index']

print(features.shape[0])

features = features.sort_values(['HADM_ID','rass_time'])
features = features.reset_index()
del features['index']

keep = []
for i in range(features.shape[0]-1):
  if features.loc[i]['HADM_ID'] == features.loc[i+1]['HADM_ID']:
    continue
  else: 
    keep.append(i)

keep.append(features.shape[0]-1)

features = features.iloc[keep]
features = features.reset_index()
del features['index']

print(features.shape[0])

52464
5043


In [ ]:
### GCS.
gcs1 = pd.read_csv('gcs1.csv', header=0)
gcs2 = pd.read_csv('gcs2.csv', header=0)
gcs3 = pd.read_csv('gcs3.csv', header=0)
gcs4 = pd.read_csv('gcs4.csv', header=0)
gcs5 = pd.read_csv('gcs5.csv', header=0)
gcs6 = pd.read_csv('gcs6.csv', header=0)
gcs7 = pd.read_csv('gcs7.csv', header=0)
gcs8 = pd.read_csv('gcs8.csv', header=0)
gcs9 = pd.read_csv('gcs9.csv', header=0)
gcs13 = pd.read_csv('gcs13.csv', header=0)
gcs14 = pd.read_csv('gcs14.csv', header=0)
gcs15 = pd.read_csv('gcs15.csv', header=0)

gcs = pd.concat([gcs1, gcs2, gcs3, gcs4, gcs5, gcs6, gcs7, gcs8, gcs9, gcs13, gcs14, gcs15])
gcs = gcs[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID','CHARTTIME', 'VALUENUM']]
gcs.columns = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID','gcs_id','gcs_time', 'gcs']
print(gcs.shape)
gcs.head()


(546751, 6)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,gcs_id,gcs_time,gcs
0,157,107880,264885.0,198,2106-06-20 07:00:00,11.0
1,156,168847,260696.0,198,2120-05-27 19:00:00,10.0
2,156,168847,260696.0,198,2120-05-28 23:00:00,10.0
3,156,168847,260696.0,198,2120-05-30 03:00:00,10.0
4,156,168847,260696.0,198,2120-05-31 02:00:00,15.0


In [ ]:
### DELETE, was used to check groupby sum was correct.
gcs_meta_id = [220739,223900,223901]
gcs_meta = gcs.iloc[np.where(gcs['gcs_id'].isin(gcs_meta_id) == True)[0]]
gcs_meta.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,gcs_id,gcs_time,gcs
0,1860,195382,272392.0,220739,2141-07-17 08:00:00,4.0
1,1860,195382,272392.0,223900,2141-07-17 08:00:00,5.0
2,1860,195382,272392.0,223901,2141-07-17 08:00:00,6.0
3,1860,195382,272392.0,220739,2141-07-17 12:00:00,3.0
4,1860,195382,272392.0,223900,2141-07-17 12:00:00,1.0


In [ ]:
### Get Metavision GCS.
gcs = gcs.groupby(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'gcs_time'], sort = False)['gcs'].sum()
gcs = gcs.reset_index()
gcs.head()
#gcs['gcs'].value_counts()
#gcs_meta_id = [220739,223900,223901]
#gcs_meta = gcs.iloc[np.where(gcs['gcs_id'].isin(gcs_meta_id) == True)[0]]
#gcs_meta.head()

#gcs[gcs['ICUSTAY_ID'] == 272392]


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,gcs_time,gcs
0,157,107880,264885.0,2106-06-20 07:00:00,11.0
1,156,168847,260696.0,2120-05-27 19:00:00,10.0
2,156,168847,260696.0,2120-05-28 23:00:00,10.0
3,156,168847,260696.0,2120-05-30 03:00:00,10.0
4,156,168847,260696.0,2120-05-31 02:00:00,15.0


In [ ]:
### Get last GCS. 
features = pd.merge(features, gcs, how = 'left', on = ['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
features = features.sort_values(['HADM_ID', 'gcs_time'])
features = features.reset_index()
del features['index']

features['gcs_time'].fillna('0', inplace = True)

drop = []
change_val = []
for i in range(features.shape[0]):
  time_cutoff = features.loc[i]['OUTTIME'] 
  var_time = features.loc[i]['gcs_time']
  if time_cutoff < var_time:
    if features.loc[i]['HADM_ID'] != features.loc[i-1]['HADM_ID'] & features.loc[i]['HADM_ID'] != features.loc[i+1]['HADM_ID']:
      change_val.append(i)
    else: 
      drop.append(i)
  else:
    continue 

for i in range(len(change_val)):
  features.loc[change_val[i],['gcs']] = 'NaN'

features = features.drop(drop)
features = features.reset_index()
del features['index']

print(features.shape[0])

features = features.sort_values(['HADM_ID','gcs_time'])
features = features.reset_index()
del features['index']

keep = []
for i in range(features.shape[0]-1):
  if features.loc[i]['HADM_ID'] == features.loc[i+1]['HADM_ID']:
    continue
  else: 
    keep.append(i)

keep.append(features.shape[0]-1)

features = features.iloc[keep]
features = features.reset_index()
del features['index']

print(features.shape[0])

286792
5043


## Intubation variables.

In [ ]:
### Vent type
vent1 = pd.read_csv('vent1.csv', header=0)
vent2 = pd.read_csv('vent2.csv', header=0)
vent3 = pd.read_csv('vent3.csv', header=0)
vent4 = pd.read_csv('vent4.csv', header=0)
vent5 = pd.read_csv('vent5.csv', header=0)
vent6 = pd.read_csv('vent6.csv', header=0)
vent7 = pd.read_csv('vent7.csv', header=0)
vent8 = pd.read_csv('vent8.csv', header=0)
vent9 = pd.read_csv('vent9.csv', header=0)
vent13 = pd.read_csv('vent13.csv', header=0)
vent14 = pd.read_csv('vent14.csv', header=0)
vent15 = pd.read_csv('vent15.csv', header=0)

vent = pd.concat([vent1,vent2,vent3,vent4,vent5,vent6,vent7,vent8,vent9,vent13,vent14,vent15])
vent = vent[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME']]
vent.columns = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID','vent_time']
print(vent.shape)
vent.head()


(245833, 4)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,vent_time
0,157,107880,264885.0,2106-06-20 04:00:00
1,156,168847,260696.0,2120-05-26 15:00:00
2,157,107880,264885.0,2106-06-21 11:00:00
3,156,168847,260696.0,2120-05-28 22:35:00
4,156,168847,260696.0,2120-05-31 03:00:00


In [ ]:
### Get days intubated and hours between extubation and transfer.

# Remove NA vent values.
vent = vent[vent['vent_time'].isna() == False]

# Change vent charttime to yyyy-mm-dd in vent_day col.
from datetime import datetime
vent_day = []
for i in range(vent.shape[0]):
  temp = datetime.strptime(vent.iloc[i]['vent_time'], '%Y-%m-%d %H:%M:%S').date()
  #print(temp)
  vent_day.append(temp)

vent['vent_day'] = vent_day

vent.head()

# Days intubated.
intub_days = vent.groupby('ICUSTAY_ID')['vent_day'].nunique()
intub_days = pd.DataFrame(intub_days)
intub_days = intub_days.reset_index()
intub_days.columns = ['ICUSTAY_ID', 'intub_days']

vent = pd.merge(vent, intub_days, how = 'left', on = ['ICUSTAY_ID'])
vent.head()


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,vent_time,vent_day,intub_days
0,157,107880,264885.0,2106-06-20 04:00:00,2106-06-20,5.0
1,156,168847,260696.0,2120-05-26 15:00:00,2120-05-26,8.0
2,157,107880,264885.0,2106-06-21 11:00:00,2106-06-21,5.0
3,156,168847,260696.0,2120-05-28 22:35:00,2120-05-28,8.0
4,156,168847,260696.0,2120-05-31 03:00:00,2120-05-31,8.0


In [ ]:
# Combine with cohort info.
feat_vent = pd.merge(cohort, vent, how = 'left', on = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])
feat_vent['vent_time'].fillna('0', inplace = True)

# Hours between extubation and transfer.
feat_vent = feat_vent.sort_values(['ICUSTAY_ID', 'vent_time'])
feat_vent = feat_vent.reset_index()
del feat_vent['index']

feat_vent = feat_vent[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'OUTTIME', 'vent_time','intub_days']]

### Write results to csv file.
# name of csv file 
filename = "extub_hrs.csv"
    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the header
    csvwriter.writerow(feat_vent.columns) 
        
    # writing the data rows 
    for i in range(feat_vent.shape[0]):
      csvwriter.writerow(feat_vent.iloc[i])


In [ ]:
# Combine with cohort info.
from datetime import datetime, date
feat_vent = pd.merge(cohort, vent, how = 'left', on = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])
feat_vent['vent_time'].fillna('0', inplace = True)

# Hours between extubation and transfer.
feat_vent = feat_vent.sort_values(['ICUSTAY_ID', 'vent_time'])
feat_vent = feat_vent.reset_index()
del feat_vent['index']

# Get hour difference.
extub_hrs = []
icustay = []
seconds_per_hour = 60*60
for i in range(feat_vent.shape[0]-1):
  if feat_vent.iloc[i]['ICUSTAY_ID'] != feat_vent.iloc[i+1]['ICUSTAY_ID']:
    if feat_vent.iloc[i]['vent_time'] == '0':
      extub_hrs.append('NaN')
    else:
      transfer = datetime.strptime(feat_vent.iloc[i]['OUTTIME'],'%Y-%m-%d %H:%M:%S')
      #print(feat_vent.iloc[i]['OUTTIME'])
      #print(feat_vent.iloc[i]['vent_time'])
      extub = datetime.strptime(feat_vent.iloc[i]['vent_time'],'%Y-%m-%d %H:%M:%S')
      diff = transfer - extub
      #print(diff.total_seconds())
      #hrs = diff.days*24 + diff.seconds/seconds_per_hour
      hrs = diff.total_seconds()/seconds_per_hour
      #print(hrs)
      extub_hrs.append(hrs)
      icustay.append(feat_vent.iloc[i]['ICUSTAY_ID'])
    if i == feat_vent.shape[0]:
      if feat_vent.iloc[i]['vent_time'] == '0':
        extub_hrs.append('NaN')  
      else: 
        transfer = datetime.strptime(feat_vent.iloc[i]['OUTTIME'],'%Y-%m-%d %H:%M:%S')
        extub = datetime.strptime(feat_vent.iloc[i]['vent_time'],'%Y-%m-%d %H:%M:%S')
        diff = transfer-extub
        hrs = diff.total_seconds()/seconds_per_hour
        extub_hrs.append(hrs)
        icustay.append(feat_vent.iloc[i]['ICUSTAY_ID'])     

extub_hrs = pd.DataFrame(extub_hrs)
icustay = pd.DataFrame(icustay)
extub = pd.concat((icustay, extub_hrs), axis=1)
extub.columns = ['ICUSTAY_ID', 'extub_hrs']

feat_vent = pd.merge(feat_vent, extub, how = 'left', on = ['ICUSTAY_ID'])

# Keep only necessary columns.
feat_vent = feat_vent[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'intub_days', 'extub_hrs']]
feat_vent = feat_vent.drop_duplicates(subset=['ICUSTAY_ID'])
print(feat_vent.shape)
feat_vent.head()


(5043, 5)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,intub_days,extub_hrs
0,28494,199133,200059,14.0,3.68611
102,2530,113580,200077,10.0,75.6667
164,61871,124231,200087,4.0,33.6692
181,11228,167826,200102,9.0,7.23889
248,47250,187834,200131,8.0,29.7436


## Final dataset.

In [ ]:
### Get complete dataset.

# Merge all features.
data = pd.merge(features, feat_vent, on = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])
print(data.shape)

# Keep only necesary columns.
data = data[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'FIRST_CAREUNIT', 'LAST_CAREUNIT', 
             'INTIME', 'OUTTIME', 'LOS', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
             'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 
             'ETHNICITY', 'GENDER','age','age_bin','COPD_hx',
             'wbc','hct','plt','cr','lac','tbili', 'inr',
             'fio2','pao2','pafio2','hr','temp','spo2','map','rass','gcs',
             'intub_days','extub_hrs']]

print(data.shape)
data.head()


(5043, 71)
(5043, 37)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,FIRST_CAREUNIT,LAST_CAREUNIT,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,ETHNICITY,GENDER,age,age_bin,COPD_hx,wbc,hct,plt,cr,lac,tbili,inr,fio2,pao2,pafio2,hr,temp,spo2,map,rass,gcs,intub_days,extub_hrs
0,68591,100016,217590,MICU,MICU,2188-05-24 13:07:20,2188-05-30 17:16:33,6.1731,2188-05-24 13:06:00,2188-05-30 17:16:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,WHITE,M,56,1,0.0,8.1,26.7,130,1.1,2,NaN,1.7,40.0,86.0,2.150000,108.0,96.800000,100.0,NaN,0.0,10.0,5.0,4.63278
1,48539,100035,245719,MICU,MICU,2115-02-22 06:52:06,2115-03-04 19:00:50,10.5061,2115-02-22 06:51:00,2115-03-19 14:37:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,REHAB/DISTINCT PART HOSP,HISPANIC OR LATINO,M,37,0,0.0,21.2,41.7,203,1.2,1,0.3,1.2,70.0,95.0,1.357143,NaN,98.100000,95.0,NaN,0.0,14.0,8.0,33.8994
2,69282,100063,241782,TSICU,MICU,2181-04-21 00:53:05,2181-04-22 16:29:17,1.6501,2181-04-20 12:00:00,2181-04-22 23:59:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,HISPANIC OR LATINO,M,46,1,0.0,9.3,40.9,183,1,NaN,0.4,1,50.0,365.0,7.300000,80.0,100.200000,96.0,NaN,-1.0,15.0,1.0,91.0525
3,42255,100087,294404,CCU,CCU,2126-11-01 17:47:40,2126-11-05 11:17:03,3.7287,2126-11-01 17:47:00,2126-11-06 05:00:00,2126-11-06 05:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,WHITE,M,57,1,0.0,8.5,44.8,215,0.7,1.3,2.4,1.5,30.0,143.0,4.766667,NaN,98.100000,81.0,NaN,-5.0,4.0,4.0,33.3536
4,19984,100132,229201,MICU,MICU,2179-03-05 20:14:43,2179-03-24 11:37:33,18.6409,2179-03-05 16:36:00,2179-03-24 11:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,BLACK/AFRICAN AMERICAN,F,74,2,0.0,10,27.6,316,1.3,1.2,0.3,1,NaN,97.0,NaN,84.0,99.199997,97.0,NaN,NaN,15.0,7.0,76.8933


In [ ]:
### Write results to csv file.
# name of csv file 
filename = "mimic_cohort.csv"
    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the header
    csvwriter.writerow(data.columns) 
        
    # writing the data rows 
    for i in range(data.shape[0]):
      csvwriter.writerow(data.iloc[i])

# Add outcome variable back in.

In [ ]:
outcome = pd.read_csv('cohort.csv', header = 0)
outcome = outcome[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'outcome']]
mimic_cohort = pd.read_csv('/content/drive/MyDrive/BMI212/mimic_cohort.csv', header = 0)

mimic_cohort = pd.merge(mimic_cohort, outcome, on = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])
mimic_cohort.head()



,SUBJECT_ID,HADM_ID,ICUSTAY_ID,FIRST_CAREUNIT,LAST_CAREUNIT,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,ETHNICITY,GENDER,age,age_bin,COPD_hx,wbc,hct,plt,cr,lac,tbili,inr,fio2,pao2,pafio2,hr,temp,spo2,map,rass,gcs,intub_days,extub_hrs,outcome
0,68591,100016,217590,MICU,MICU,2188-05-24 13:07:20,2188-05-30 17:16:33,6.1731,2188-05-24 13:06:00,2188-05-30 17:16:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,WHITE,M,56,1,0.0,8.1,26.7,130.0,1.1,2.0,NaN,1.7,40.0,86.0,2.150000,108.0,96.800000,100.0,NaN,0.0,10.0,5.0,4.632778,0.0
1,48539,100035,245719,MICU,MICU,2115-02-22 06:52:06,2115-03-04 19:00:50,10.5061,2115-02-22 06:51:00,2115-03-19 14:37:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,REHAB/DISTINCT PART HOSP,HISPANIC OR LATINO,M,37,0,0.0,21.2,41.7,203.0,1.2,1.0,0.3,1.2,70.0,95.0,1.357143,NaN,98.100000,95.0,NaN,0.0,14.0,8.0,33.899444,0.0
2,69282,100063,241782,TSICU,MICU,2181-04-21 00:53:05,2181-04-22 16:29:17,1.6501,2181-04-20 12:00:00,2181-04-22 23:59:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,HISPANIC OR LATINO,M,46,1,0.0,9.3,40.9,183.0,1.0,NaN,0.4,1.0,50.0,365.0,7.300000,80.0,100.200000,96.0,NaN,-1.0,15.0,1.0,91.052500,0.0
3,42255,100087,294404,CCU,CCU,2126-11-01 17:47:40,2126-11-05 11:17:03,3.7287,2126-11-01 17:47:00,2126-11-06 05:00:00,2126-11-06 05:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,WHITE,M,57,1,0.0,8.5,44.8,215.0,0.7,1.3,2.4,1.5,30.0,143.0,4.766667,NaN,98.100000,81.0,NaN,-5.0,4.0,4.0,33.353611,0.0
4,19984,100132,229201,MICU,MICU,2179-03-05 20:14:43,2179-03-24 11:37:33,18.6409,2179-03-05 16:36:00,2179-03-24 11:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,BLACK/AFRICAN AMERICAN,F,74,2,0.0,10.0,27.6,316.0,1.3,1.2,0.3,1.0,NaN,97.0,NaN,84.0,99.199997,97.0,NaN,NaN,15.0,7.0,76.893333,0.0


In [ ]:
### Write results to csv file.
# name of csv file 
filename = "mimic_cohort.csv"
    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the header
    csvwriter.writerow(mimic_cohort.columns) 

    # writing the data rows 
    for i in range(mimic_cohort.shape[0]):
      csvwriter.writerow(mimic_cohort.iloc[i])

In [ ]:
mimic_cohort['ADMITTIME']

0       2188-05-24 13:06:00
1       2115-02-22 06:51:00
2       2181-04-20 12:00:00
3       2126-11-01 17:47:00
4       2179-03-05 16:36:00
               ...         
5038    2184-01-01 20:23:00
5039    2195-10-08 22:51:00
5040    2183-03-24 03:13:00
5041    2181-05-15 15:26:00
5042    2188-07-07 18:47:00
Name: ADMITTIME, Length: 5043, dtype: object

In [ ]:
### Demographic distribution.

readmit_F = int(len(cohort_readmit_final[cohort_readmit_final['GENDER'] == 'F'])/2)
readmit_M = int(len(cohort_readmit_final[cohort_readmit_final['GENDER'] == 'M'])/2)

print('Readmit: ', '\n', 'Total: ', len(cohort_readmit_final['SUBJECT_ID'].unique()), '\n', 'Female: ', readmit_F, '\n', 
      'Male: ', readmit_M, '\n', '\n')

nonreadmit_F = len(cohort_nonreadmit_final[cohort_nonreadmit_final['GENDER'] == 'F'])
nonreadmit_M = len(cohort_nonreadmit_final[cohort_nonreadmit_final['GENDER'] == 'M'])

print('Non readmit: ', '\n', 'Total: ', len(cohort_nonreadmit_final['SUBJECT_ID'].unique()), '\n', 'Female: ', nonreadmit_F, '\n', 
      'Male: ', nonreadmit_M, '\n', '\n')


readmit_C = int(len(cohort_readmit_final[cohort_readmit_final['ETHNICITY'] == 'WHITE'])/2)
readmit_AA = int(len(cohort_readmit_final[cohort_readmit_final['ETHNICITY'] == 'BLACK/AFRICAN AMERICAN'])/2)
readmit_H = int(len(cohort_readmit_final[cohort_readmit_final['ETHNICITY'].str.contains('HISPANIC')])/2)
readmit_A = int(len(cohort_readmit_final[cohort_readmit_final['ETHNICITY'] == 'ASIAN'])/2)
readmit_U = int(cohort_readmit_final.shape[0]/2 - readmit_C - readmit_AA - readmit_H - readmit_A)

print('Readmit: ', '\n', 'Total: ', cohort_readmit_final['SUBJECT_ID'].nunique(), '\n', 'Caucasian: ', readmit_C, '\n', 
      'African American: ', readmit_AA, '\n', 'Latino/a: ', readmit_H, '\n', 'Asian: ', readmit_A, '\n', 'Unknown: ',
      readmit_U, '\n', '\n')

nonreadmit_C = len(cohort_nonreadmit_final[cohort_nonreadmit_final['ETHNICITY'] == 'WHITE'])
nonreadmit_AA = len(cohort_nonreadmit_final[cohort_nonreadmit_final['ETHNICITY'] == 'BLACK/AFRICAN AMERICAN'])
nonreadmit_H = len(cohort_nonreadmit_final[cohort_nonreadmit_final['ETHNICITY'].str.contains('HISPANIC')])
nonreadmit_A = len(cohort_nonreadmit_final[cohort_nonreadmit_final['ETHNICITY'] == 'ASIAN'])
nonreadmit_U = cohort_nonreadmit_final.shape[0] - nonreadmit_C - nonreadmit_AA - nonreadmit_H - nonreadmit_A

print('Non readmit: ', '\n', 'Total: ', cohort_nonreadmit_final.shape[0], '\n', 'Caucasian: ', nonreadmit_C, '\n', 
      'African American: ', nonreadmit_AA, '\n', 'Latino/a: ', nonreadmit_H, '\n', 'Asian: ', nonreadmit_A, '\n', 
      'Unknown: ',nonreadmit_U, '\n', '\n')



Readmit:  
 Total:  42 
 Female:  21 
 Male:  21 
 

Non readmit:  
 Total:  855 
 Female:  418 
 Male:  437 
 

Readmit:  
 Total:  42 
 Caucasian:  32 
 African American:  4 
 Latino/a:  1 
 Asian:  0 
 Unknown:  5 
 

Non readmit:  
 Total:  855 
 Caucasian:  592 
 African American:  98 
 Latino/a:  28 
 Asian:  19 
 Unknown:  118 
 



# Create matching features of STARR

**STARR: MIMIC**  
"id" = "SUBJECT_ID"  
"readmit" = ?   
"total_los" = "OUTTIME" - "ADMITTIME"    
"icu_los" = "OUTTIME" - "INTIME"  
"intub_days" = "intub_days"  
"age" = age"  
"female" = "GENDER" (convert to 0 or 1)  
"race" = "ETHNICITY" (re-group)   
"ED_admit" = "ADMISSION_TYPE"  
"COPD_hx" = "COPD_hx"  
"hct" = "hct"  
"wbc" = "wbc"  
"inr" = "inr"  
"cr" = "cr"  
"plt" = "plt"  
"sao2" = "spo2" (rename)  
"hr" = "hr"  
"map" = "map"  
"gcs" = "gcs"  
"lactate" = "lac" (rename)  
"rass" = "rass"  
"temp_f" = "temp" (rename)  
"pao2" = "pao2"  
"pafio2" = "pafio2"  


In [ ]:
import pandas as pd

mimic_file = "mimic_cohort.csv"
data = pd.read_csv(mimic_file)

In [ ]:
for ethnicity in pd.unique(data["ETHNICITY"]):
  if "ASIAN" in ethnicity:
    print("%s: Asian" % ethnicity)
  elif "BLACK" in ethnicity:
    print("%s: Black" % ethnicity)
  elif "HISPANIC" in ethnicity or "LATINO" in ethnicity or "SOUTH AMERICA" in ethnicity:
    print("%s: Hispanic/Latino" % ethnicity)
  elif "NATIVE" in ethnicity and "AMERICAN" in ethnicity:
    print("%s: Native American" % ethnicity)
  elif "PACIFIC ISLANDER" in ethnicity:
    print("%s: Pacific Islander" % ethnicity)
  elif "UNKNOWN" in ethnicity or "UNABLE" in ethnicity or "DECLINE" in ethnicity:
    print("%s: Unknown" % ethnicity)
  elif "WHITE" in ethnicity or "MIDDLE EAST" in ethnicity or "PORTUGUESE" in ethnicity:
    print("%s: White" % ethnicity)
  else:
    print("%s: Other" % ethnicity)

WHITE: White
HISPANIC OR LATINO: Hispanic/Latino
BLACK/AFRICAN AMERICAN: Black
OTHER: Other
UNKNOWN/NOT SPECIFIED: Unknown
UNABLE TO OBTAIN: Unknown
ASIAN: Asian
WHITE - BRAZILIAN: White
PATIENT DECLINED TO ANSWER: Unknown
BLACK/HAITIAN: Black
ASIAN - CHINESE: Asian
WHITE - RUSSIAN: White
MULTI RACE ETHNICITY: Other
BLACK/CAPE VERDEAN: Black
ASIAN - VIETNAMESE: Asian
BLACK/AFRICAN: Black
WHITE - OTHER EUROPEAN: White
PORTUGUESE: White
NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER: Pacific Islander
HISPANIC/LATINO - PUERTO RICAN: Hispanic/Latino
MIDDLE EASTERN: White
ASIAN - ASIAN INDIAN: Asian
HISPANIC/LATINO - DOMINICAN: Hispanic/Latino
AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE: Native American
ASIAN - OTHER: Asian
ASIAN - KOREAN: Asian
ASIAN - FILIPINO: Asian
AMERICAN INDIAN/ALASKA NATIVE: Native American
ASIAN - JAPANESE: Asian
HISPANIC/LATINO - CENTRAL AMERICAN (OTHER): Hispanic/Latino
HISPANIC/LATINO - COLOMBIAN: Hispanic/Latino
HISPANIC/LATINO - MEXICAN: Hispanic/Latin

In [ ]:
print(data.shape)
print(len(pd.unique(data["HADM_ID"])))
print(len(pd.unique(data["SUBJECT_ID"])))
print(len(pd.unique(data["ICUSTAY_ID"])))

(5043, 38)
5043
5043
5043


In [ ]:
# rename some columns to match those from STARR
data.rename(columns={"SUBJECT_ID": "id", "spo2": "sao2", "lac": "lactate", "temp": "temp_f", "outcome": "readmit"}, inplace=True)

# convert descriptive string to numeric string 
data["female"] = (data["GENDER"] == "F") * 1
data["ED_admit"] = (data["ADMISSION_LOCATION"] == "EMERGENCY ROOM ADMIT") * 1

# re-group ETHNICITY column into same race categories
race = []
for ethnicity in data["ETHNICITY"]:
  if "ASIAN" in ethnicity:
    race.append("Asian")
  elif "BLACK" in ethnicity:
    race.append("Black")
  elif "HISPANIC" in ethnicity or "LATINO" in ethnicity or "SOUTH AMERICA" in ethnicity:
    race.append("Hispanic/Latino")
  elif "NATIVE" in ethnicity and "AMERICAN" in ethnicity:
    race.append("Native American")
  elif "PACIFIC ISLANDER" in ethnicity:
    race.append("Pacific Islander")
  elif "UNKNOWN" in ethnicity or "UNABLE" in ethnicity or "DECLINE" in ethnicity:
    race.append("Unknown")
  elif "WHITE" in ethnicity or "MIDDLE EAST" in ethnicity or "PORTUGUESE" in ethnicity:
    race.append("White")
  else:
    race.append("Other")
data["race"] = race

# calculate LOS
def calculate_los_days(data, start_col, end_col):
  los_dt = pd.to_datetime(data[end_col]) - pd.to_datetime(data[start_col])
  los = []
  for dt in los_dt:
    los.append(dt.days)
  
  return los

data["icu_los"] = calculate_los_days(data, "INTIME", "OUTTIME")
data["total_los"] = calculate_los_days(data, "ADMITTIME", "OUTTIME")

In [ ]:
print(data.columns)
data.head()

Index(['id', 'HADM_ID', 'ICUSTAY_ID', 'FIRST_CAREUNIT', 'LAST_CAREUNIT',
       'INTIME', 'OUTTIME', 'LOS', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'ETHNICITY', 'GENDER', 'age', 'age_bin', 'COPD_hx', 'wbc', 'hct', 'plt',
       'cr', 'lactate', 'tbili', 'inr', 'fio2', 'pao2', 'pafio2', 'hr',
       'temp_f', 'sao2', 'map', 'rass', 'gcs', 'intub_days', 'extub_hrs',
       'readmit', 'female', 'ED_admit', 'race', 'icu_los', 'total_los'],
      dtype='object')


,id,HADM_ID,ICUSTAY_ID,FIRST_CAREUNIT,LAST_CAREUNIT,INTIME,OUTTIME,LOS,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,ETHNICITY,GENDER,age,age_bin,COPD_hx,wbc,hct,plt,cr,lactate,tbili,inr,fio2,pao2,pafio2,hr,temp_f,sao2,map,rass,gcs,intub_days,extub_hrs,readmit,female,ED_admit,race,icu_los,total_los
0,68591,100016,217590,MICU,MICU,2188-05-24 13:07:20,2188-05-30 17:16:33,6.1731,2188-05-24 13:06:00,2188-05-30 17:16:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,WHITE,M,56,1,0.0,8.1,26.7,130.0,1.1,2.0,NaN,1.7,40.0,86.0,2.150000,108.0,96.800000,100.0,NaN,0.0,10.0,5.0,4.632778,0.0,0,0,White,6,6
1,48539,100035,245719,MICU,MICU,2115-02-22 06:52:06,2115-03-04 19:00:50,10.5061,2115-02-22 06:51:00,2115-03-19 14:37:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,REHAB/DISTINCT PART HOSP,HISPANIC OR LATINO,M,37,0,0.0,21.2,41.7,203.0,1.2,1.0,0.3,1.2,70.0,95.0,1.357143,NaN,98.100000,95.0,NaN,0.0,14.0,8.0,33.899444,0.0,0,0,Hispanic/Latino,10,10
2,69282,100063,241782,TSICU,MICU,2181-04-21 00:53:05,2181-04-22 16:29:17,1.6501,2181-04-20 12:00:00,2181-04-22 23:59:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,HISPANIC OR LATINO,M,46,1,0.0,9.3,40.9,183.0,1.0,NaN,0.4,1.0,50.0,365.0,7.300000,80.0,100.200000,96.0,NaN,-1.0,15.0,1.0,91.052500,0.0,0,1,Hispanic/Latino,1,2
3,42255,100087,294404,CCU,CCU,2126-11-01 17:47:40,2126-11-05 11:17:03,3.7287,2126-11-01 17:47:00,2126-11-06 05:00:00,2126-11-06 05:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,WHITE,M,57,1,0.0,8.5,44.8,215.0,0.7,1.3,2.4,1.5,30.0,143.0,4.766667,NaN,98.100000,81.0,NaN,-5.0,4.0,4.0,33.353611,0.0,0,0,White,3,3
4,19984,100132,229201,MICU,MICU,2179-03-05 20:14:43,2179-03-24 11:37:33,18.6409,2179-03-05 16:36:00,2179-03-24 11:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,LONG TERM CARE HOSPITAL,BLACK/AFRICAN AMERICAN,F,74,2,0.0,10.0,27.6,316.0,1.3,1.2,0.3,1.0,NaN,97.0,NaN,84.0,99.199997,97.0,NaN,NaN,15.0,7.0,76.893333,0.0,1,1,Black,18,18


In [ ]:
cols_keep = ["id", "readmit", "total_los", "icu_los", "intub_days", "age", "female", "race", "ED_admit", "COPD_hx",
             "hct", "wbc", "inr", "cr", "plt", "sao2", "hr", "map", "gcs", "lactate", "rass", "temp_f", "pao2", "pafio2"]
is_subset = set(cols_keep).issubset(set(data.columns))
if not is_subset:
  print(set(cols_keep).difference(set(data.columns)))
else:
  print("all columns present")

all columns present


In [ ]:
data.to_csv("mimic_cohort_converted.csv", index=False)